# Importando bases

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier

In [2]:
df_bolsonaro_train = pd.read_excel('Bolsonaro_treino_sem_stop.xlsx')  
df_lula_train = pd.read_excel('Lula_treino_sem_stop.xlsx') 
df_simone_train = pd.read_excel('Simone_treino_sem_stop.xlsx') 
df_ciro_train = pd.read_excel('Ciro_treino_sem_stop.xlsx') 

In [31]:
df_bolsonaro = pd.read_excel('Bolsonaro_todos_sem_stop.xlsx', index_col=0)
df_lula = pd.read_excel('Lula_todos_sem_stop.xlsx', index_col=0)
df_simone = pd.read_excel('Simone_todos_sem_stop.xlsx', index_col=0)
df_ciro = pd.read_excel('Ciro_todos_sem_stop.xlsx', index_col=0)



In [3]:
df_lula_train = df_lula_train[df_lula_train['Sentimento']!=' ']

In [4]:
# Bag Of Words

def Bag_of_words(df):
    matrix = CountVectorizer()
    text = []
    for i in df.index:
        text.append("".join(df['novo_texto'][i]))
    X = matrix.fit_transform(text)
    return X, matrix

def Bag_of_words_teste(df, matrix):
    text = []
    for i in df.index:
        text.append("".join(df['novo_texto'][i]))
    X = matrix.transform(text)
    return X

def tfidf(df):
    matrix = TfidfVectorizer()
    text = []
    for i in df.index:
        text.append("".join(df['novo_texto'][i]))
    X = matrix.fit_transform(text)
    return X, matrix

def tfidf_teste(df, matrix):
    text = []
    for i in df.index:
        text.append("".join(df['novo_texto'][i]))
    X = matrix.transform(text)
    return X



## Base treino bag of words

In [5]:
X_bw_bolsonaro_train, bw_bolsonaro = Bag_of_words(df_bolsonaro_train)
X_bw_lula_train, bw_lula = Bag_of_words(df_lula_train)
X_bw_simone_train, bw_simone = Bag_of_words(df_simone_train)
X_bw_ciro_train, bw_ciro = Bag_of_words(df_ciro_train)

## Base treino tfidf

In [6]:
X_tfidf_bolsonaro_train, tfidf_bolsonaro = tfidf(df_bolsonaro_train)
X_tfidf_lula_train, tfidf_lula = tfidf(df_lula_train)
X_tfidf_simone_train, tfidf_simone = tfidf(df_simone_train)
X_tfidf_ciro_train, tfidf_ciro = tfidf(df_ciro_train)

# Predict treino

In [7]:
Y_bolsonaro_train = df_bolsonaro_train['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int)
Y_lula_train = df_lula_train['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int)
Y_simone_train = df_simone_train['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int)
Y_ciro_train = df_ciro_train['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int)

In [32]:
from sklearn.model_selection import train_test_split

## Bolsonaro

### Bolsonaro - Arvore de Decisao - bw

In [29]:
X_bw_bolsonaro_train2 = X_bw_bolsonaro_train

tree_bolsonaro_bw_test = tree.DecisionTreeClassifier()
tree_X_bw_bolsonaro_train, tree_X_bw_bolsonaro_test, tree_Y_bw_bolsonaro_train, tree_Y_bw_bolsonaro_test = train_test_split(X_bw_bolsonaro_train2, Y_bolsonaro_train, test_size=0.2)
tree_bolsonaro_bw_test = tree_bolsonaro_bw_test.fit(tree_X_bw_bolsonaro_train, tree_Y_bw_bolsonaro_train)

score_tree_bw = tree_bolsonaro_bw_test.score(tree_X_bw_bolsonaro_test, tree_Y_bw_bolsonaro_test)
tree_bolsonaro_bw = tree_bolsonaro_bw_test
for i in range(0,100):
    tree_bolsonaro_bw_test = tree.DecisionTreeClassifier()
    tree_X_bw_bolsonaro_train, tree_X_bw_bolsonaro_test, tree_Y_bw_bolsonaro_train, tree_Y_bw_bolsonaro_test = train_test_split(X_bw_bolsonaro_train2, Y_bolsonaro_train, test_size=0.2)
    tree_bolsonaro_bw_test = tree_bolsonaro_bw_test.fit(tree_X_bw_bolsonaro_train, tree_Y_bw_bolsonaro_train)
    score_tree_bw_test = tree_bolsonaro_bw_test.score(tree_X_bw_bolsonaro_test, tree_Y_bw_bolsonaro_test)
    if(score_tree_bw_test>score_tree_bw):
        score_tree_bw = score_tree_bw_test
        tree_bolsonaro_bw = tree_bolsonaro_bw_test
score_tree_bw

0.8087876322213181

In [33]:
i1=0
for i in range(5000,len(df_bolsonaro),5000):
    if (i==5000):
        predict_bolsonaro_bw_tree = tree_bolsonaro_bw.predict(Bag_of_words_teste(df_bolsonaro[i1:i], bw_bolsonaro).toarray())
    else:
        predict_bolsonaro_bw_tree = np.append(predict_bolsonaro_bw_tree, tree_bolsonaro_bw.predict(Bag_of_words_teste(df_bolsonaro[i1:i], bw_bolsonaro).toarray()))
    i1=i

predict_bolsonaro_bw_tree = np.append(predict_bolsonaro_bw_tree,tree_bolsonaro_bw.predict(Bag_of_words_teste(df_bolsonaro[i1:], bw_bolsonaro).toarray()))


df_bolsonaro['predict_bw_ArvoreDeDecisao'] = predict_bolsonaro_bw_tree

np.unique(predict_bolsonaro_bw_tree, return_counts=True)

(array([-1,  0,  1]), array([113610,  11239, 821589], dtype=int64))

### Bolsonaro - Arvore de Decisao - tfidf

In [34]:
X_tfidf_bolsonaro_train2 = X_tfidf_bolsonaro_train

tree_bolsonaro_tfidf_test = tree.DecisionTreeClassifier()
tree_X_tfidf_bolsonaro_train, tree_X_tfidf_bolsonaro_test, tree_Y_tfidf_bolsonaro_train, tree_Y_tfidf_bolsonaro_test = train_test_split(X_tfidf_bolsonaro_train2, Y_bolsonaro_train, test_size=0.2)
tree_bolsonaro_tfidf_test = tree_bolsonaro_tfidf_test.fit(tree_X_tfidf_bolsonaro_train, tree_Y_tfidf_bolsonaro_train)

score_tree_tfidf = tree_bolsonaro_tfidf_test.score(tree_X_tfidf_bolsonaro_test, tree_Y_tfidf_bolsonaro_test)
tree_bolsonaro_tfidf = tree_bolsonaro_tfidf_test
for i in range(0,100):
    tree_bolsonaro_tfidf_test = tree.DecisionTreeClassifier()
    tree_X_tfidf_bolsonaro_train, tree_X_tfidf_bolsonaro_test, tree_Y_tfidf_bolsonaro_train, tree_Y_tfidf_bolsonaro_test = train_test_split(X_tfidf_bolsonaro_train2, Y_bolsonaro_train, test_size=0.2)
    tree_bolsonaro_tfidf_test = tree_bolsonaro_tfidf_test.fit(tree_X_tfidf_bolsonaro_train, tree_Y_tfidf_bolsonaro_train)
    score_tree_tfidf_test = tree_bolsonaro_tfidf_test.score(tree_X_tfidf_bolsonaro_test, tree_Y_tfidf_bolsonaro_test)
    if(score_tree_tfidf_test>score_tree_tfidf):
        score_tree_tfidf = score_tree_tfidf_test
        tree_bolsonaro_tfidf = tree_bolsonaro_tfidf_test
score_tree_tfidf

0.8144833197721725

In [35]:
i1=0
for i in range(5000,len(df_bolsonaro),5000):
    if (i==5000):
        predict_bolsonaro_tfidf_tree = tree_bolsonaro_tfidf.predict(tfidf_teste(df_bolsonaro[i1:i], tfidf_bolsonaro).toarray())
    else:
        predict_bolsonaro_tfidf_tree = np.append(predict_bolsonaro_tfidf_tree, tree_bolsonaro_tfidf.predict(tfidf_teste(df_bolsonaro[i1:i], tfidf_bolsonaro).toarray()))
    i1=i

predict_bolsonaro_tfidf_tree = np.append(predict_bolsonaro_tfidf_tree,tree_bolsonaro_tfidf.predict(tfidf_teste(df_bolsonaro[i1:], tfidf_bolsonaro).toarray()))


df_bolsonaro['predict_tfidf_ArvoreDeDecisao'] = predict_bolsonaro_tfidf_tree

np.unique(predict_bolsonaro_tfidf_tree, return_counts=True)

(array([-1,  0,  1]), array([153802,   9487, 783149], dtype=int64))

### Bolsonaro - Naive Bayes - bw

In [38]:
GaussianNB_bolsonaro_bw_test = GaussianNB()
GaussianNB_X_bw_bolsonaro_train, GaussianNB_X_bw_bolsonaro_test, GaussianNB_Y_bw_bolsonaro_train, GaussianNB_Y_bw_bolsonaro_test = train_test_split(X_bw_bolsonaro_train2, Y_bolsonaro_train, test_size=0.2)
GaussianNB_bolsonaro_bw_test = GaussianNB_bolsonaro_bw_test.fit(GaussianNB_X_bw_bolsonaro_train.toarray(), GaussianNB_Y_bw_bolsonaro_train)

score_GaussianNB_bw = GaussianNB_bolsonaro_bw_test.score(GaussianNB_X_bw_bolsonaro_test.toarray(), GaussianNB_Y_bw_bolsonaro_test)
GaussianNB_bolsonaro_bw = GaussianNB_bolsonaro_bw_test
for i in range(0,100):
    GaussianNB_bolsonaro_bw_test = GaussianNB()
    GaussianNB_X_bw_bolsonaro_train, GaussianNB_X_bw_bolsonaro_test, GaussianNB_Y_bw_bolsonaro_train, GaussianNB_Y_bw_bolsonaro_test = train_test_split(X_bw_bolsonaro_train2, Y_bolsonaro_train, test_size=0.2)
    GaussianNB_bolsonaro_bw_test = GaussianNB_bolsonaro_bw_test.fit(GaussianNB_X_bw_bolsonaro_train.toarray(), GaussianNB_Y_bw_bolsonaro_train)
    score_GaussianNB_bw_test = GaussianNB_bolsonaro_bw_test.score(GaussianNB_X_bw_bolsonaro_test.toarray(), GaussianNB_Y_bw_bolsonaro_test)
    if(score_GaussianNB_bw_test>score_GaussianNB_bw):
        score_GaussianNB_bw = score_GaussianNB_bw_test
        GaussianNB_bolsonaro_bw = GaussianNB_bolsonaro_bw_test
score_GaussianNB_bw

0.6403580146460537

In [39]:
i1=0
for i in range(5000,len(df_bolsonaro),5000):
    if (i==5000):
        predict_bolsonaro_bw_nb = GaussianNB_bolsonaro_bw.predict(Bag_of_words_teste(df_bolsonaro[i1:i], bw_bolsonaro).toarray())
    else:
        predict_bolsonaro_bw_nb = np.append(predict_bolsonaro_bw_nb, GaussianNB_bolsonaro_bw.predict(Bag_of_words_teste(df_bolsonaro[i1:i], bw_bolsonaro).toarray()))
    i1=i

predict_bolsonaro_bw_nb = np.append(predict_bolsonaro_bw_nb,GaussianNB_bolsonaro_bw.predict(Bag_of_words_teste(df_bolsonaro[i1:], bw_bolsonaro).toarray()))

df_bolsonaro['predict_bw_NaiveBayes'] = predict_bolsonaro_bw_nb

np.unique(predict_bolsonaro_bw_nb, return_counts=True)

(array([-1,  0,  1]), array([299740,  14600, 632098], dtype=int64))

### Bolsonaro - Naive Bayes - tfidf

In [40]:
GaussianNB_bolsonaro_tfidf_test = GaussianNB()
GaussianNB_X_tfidf_bolsonaro_train, GaussianNB_X_tfidf_bolsonaro_test, GaussianNB_Y_tfidf_bolsonaro_train, GaussianNB_Y_tfidf_bolsonaro_test = train_test_split(X_tfidf_bolsonaro_train2, Y_bolsonaro_train, test_size=0.2)
GaussianNB_bolsonaro_tfidf_test = GaussianNB_bolsonaro_tfidf_test.fit(GaussianNB_X_tfidf_bolsonaro_train.toarray(), GaussianNB_Y_tfidf_bolsonaro_train)

score_GaussianNB_tfidf = GaussianNB_bolsonaro_tfidf_test.score(GaussianNB_X_tfidf_bolsonaro_test.toarray(), GaussianNB_Y_tfidf_bolsonaro_test)
GaussianNB_bolsonaro_tfidf = GaussianNB_bolsonaro_tfidf_test
for i in range(0,100):
    GaussianNB_bolsonaro_tfidf_test = GaussianNB()
    GaussianNB_X_tfidf_bolsonaro_train, GaussianNB_X_tfidf_bolsonaro_test, GaussianNB_Y_tfidf_bolsonaro_train, GaussianNB_Y_tfidf_bolsonaro_test = train_test_split(X_tfidf_bolsonaro_train2, Y_bolsonaro_train, test_size=0.2)
    GaussianNB_bolsonaro_tfidf_test = GaussianNB_bolsonaro_tfidf_test.fit(GaussianNB_X_tfidf_bolsonaro_train.toarray(), GaussianNB_Y_tfidf_bolsonaro_train)
    score_GaussianNB_tfidf_test = GaussianNB_bolsonaro_tfidf_test.score(GaussianNB_X_tfidf_bolsonaro_test.toarray(), GaussianNB_Y_tfidf_bolsonaro_test)
    if(score_GaussianNB_tfidf_test>score_GaussianNB_tfidf):
        score_GaussianNB_tfidf = score_GaussianNB_tfidf_test
        GaussianNB_bolsonaro_tfidf = GaussianNB_bolsonaro_tfidf_test
score_GaussianNB_tfidf

0.644426362896664

In [41]:
i1=0
for i in range(5000,len(df_bolsonaro),5000):
    if (i==5000):
        predict_bolsonaro_tfidf_nb = GaussianNB_bolsonaro_tfidf.predict(tfidf_teste(df_bolsonaro[i1:i], tfidf_bolsonaro).toarray())
    else:
        predict_bolsonaro_tfidf_nb = np.append(predict_bolsonaro_tfidf_nb, GaussianNB_bolsonaro_tfidf.predict(tfidf_teste(df_bolsonaro[i1:i], tfidf_bolsonaro).toarray()))
    i1=i

predict_bolsonaro_tfidf_nb = np.append(predict_bolsonaro_tfidf_nb,GaussianNB_bolsonaro_tfidf.predict(tfidf_teste(df_bolsonaro[i1:], tfidf_bolsonaro).toarray()))

df_bolsonaro['predict_tfidf_NaiveBayes'] = predict_bolsonaro_tfidf_nb

np.unique(predict_bolsonaro_tfidf_nb, return_counts=True)

(array([-1,  0,  1]), array([295271,  17166, 634001], dtype=int64))

### Bolsonaro - Random Forest - bw

In [43]:
RandomForest_bolsonaro_bw_test = RandomForestClassifier()
RandomForest_X_bw_bolsonaro_train, RandomForest_X_bw_bolsonaro_test, RandomForest_Y_bw_bolsonaro_train, RandomForest_Y_bw_bolsonaro_test = train_test_split(X_bw_bolsonaro_train2, Y_bolsonaro_train, test_size=0.2)
RandomForest_bolsonaro_bw_test = RandomForest_bolsonaro_bw_test.fit(RandomForest_X_bw_bolsonaro_train.toarray(), RandomForest_Y_bw_bolsonaro_train)

score_RandomForest_bw = RandomForest_bolsonaro_bw_test.score(RandomForest_X_bw_bolsonaro_test.toarray(), RandomForest_Y_bw_bolsonaro_test)
RandomForest_bolsonaro_bw = RandomForest_bolsonaro_bw_test
for i in range(0,100):
    RandomForest_bolsonaro_bw_test = RandomForestClassifier()
    RandomForest_X_bw_bolsonaro_train, RandomForest_X_bw_bolsonaro_test, RandomForest_Y_bw_bolsonaro_train, RandomForest_Y_bw_bolsonaro_test = train_test_split(X_bw_bolsonaro_train2, Y_bolsonaro_train, test_size=0.2)
    RandomForest_bolsonaro_bw_test = RandomForest_bolsonaro_bw_test.fit(RandomForest_X_bw_bolsonaro_train.toarray(), RandomForest_Y_bw_bolsonaro_train)
    score_RandomForest_bw_test = RandomForest_bolsonaro_bw_test.score(RandomForest_X_bw_bolsonaro_test.toarray(), RandomForest_Y_bw_bolsonaro_test)
    if(score_RandomForest_bw_test>score_RandomForest_bw):
        score_RandomForest_bw = score_RandomForest_bw_test
        RandomForest_bolsonaro_bw = RandomForest_bolsonaro_bw_test
score_RandomForest_bw

0.8665581773799838

In [44]:
i1=0
for i in range(5000,len(df_bolsonaro),5000):
    if (i==5000):
        predict_bolsonaro_bw_rf = RandomForest_bolsonaro_bw.predict(Bag_of_words_teste(df_bolsonaro[i1:i], bw_bolsonaro).toarray())
    else:
        predict_bolsonaro_bw_rf = np.append(predict_bolsonaro_bw_rf, RandomForest_bolsonaro_bw.predict(Bag_of_words_teste(df_bolsonaro[i1:i], bw_bolsonaro).toarray()))
    i1=i

predict_bolsonaro_bw_rf = np.append(predict_bolsonaro_bw_rf,RandomForest_bolsonaro_bw.predict(Bag_of_words_teste(df_bolsonaro[i1:], bw_bolsonaro).toarray()))

df_bolsonaro['predict_bw_RandomForest'] = predict_bolsonaro_bw_rf

np.unique(predict_bolsonaro_bw_rf, return_counts=True)

(array([-1,  0,  1]), array([ 22207,   2649, 921582], dtype=int64))

### Bolsonaro - Random Forest - tfidf

In [45]:
RandomForest_bolsonaro_tfidf_test = RandomForestClassifier()
RandomForest_X_tfidf_bolsonaro_train, RandomForest_X_tfidf_bolsonaro_test, RandomForest_Y_tfidf_bolsonaro_train, RandomForest_Y_tfidf_bolsonaro_test = train_test_split(X_tfidf_bolsonaro_train2, Y_bolsonaro_train, test_size=0.2)
RandomForest_bolsonaro_tfidf_test = RandomForest_bolsonaro_tfidf_test.fit(RandomForest_X_tfidf_bolsonaro_train.toarray(), RandomForest_Y_tfidf_bolsonaro_train)

score_RandomForest_tfidf = RandomForest_bolsonaro_tfidf_test.score(RandomForest_X_tfidf_bolsonaro_test.toarray(), RandomForest_Y_tfidf_bolsonaro_test)
RandomForest_bolsonaro_tfidf = RandomForest_bolsonaro_tfidf_test
for i in range(0,100):
    RandomForest_bolsonaro_tfidf_test = RandomForestClassifier()
    RandomForest_X_tfidf_bolsonaro_train, RandomForest_X_tfidf_bolsonaro_test, RandomForest_Y_tfidf_bolsonaro_train, RandomForest_Y_tfidf_bolsonaro_test = train_test_split(X_tfidf_bolsonaro_train2, Y_bolsonaro_train, test_size=0.2)
    RandomForest_bolsonaro_tfidf_test = RandomForest_bolsonaro_tfidf_test.fit(RandomForest_X_tfidf_bolsonaro_train.toarray(), RandomForest_Y_tfidf_bolsonaro_train)
    score_RandomForest_tfidf_test = RandomForest_bolsonaro_tfidf_test.score(RandomForest_X_tfidf_bolsonaro_test.toarray(), RandomForest_Y_tfidf_bolsonaro_test)
    if(score_RandomForest_tfidf_test>score_RandomForest_tfidf):
        score_RandomForest_tfidf = score_RandomForest_tfidf_test
        RandomForest_bolsonaro_tfidf = RandomForest_bolsonaro_tfidf_test
score_RandomForest_tfidf

0.8722538649308381

In [46]:
i1=0
for i in range(5000,len(df_bolsonaro),5000):
    if (i==5000):
        predict_bolsonaro_tfidf_rf = RandomForest_bolsonaro_tfidf.predict(tfidf_teste(df_bolsonaro[i1:i], tfidf_bolsonaro).toarray())
    else:
        predict_bolsonaro_tfidf_rf = np.append(predict_bolsonaro_tfidf_rf, RandomForest_bolsonaro_tfidf.predict(tfidf_teste(df_bolsonaro[i1:i], tfidf_bolsonaro).toarray()))
    i1=i

predict_bolsonaro_tfidf_rf = np.append(predict_bolsonaro_tfidf_rf,RandomForest_bolsonaro_tfidf.predict(tfidf_teste(df_bolsonaro[i1:], tfidf_bolsonaro).toarray()))

df_bolsonaro['predict_tfidf_RandomForest'] = predict_bolsonaro_tfidf_rf

np.unique(predict_bolsonaro_tfidf_rf, return_counts=True)

(array([-1,  0,  1]), array([ 20301,   3879, 922258], dtype=int64))

### Bolsonaro - SVM - bw

In [47]:
svc_bolsonaro_bw_test = LinearSVC()
svc_X_bw_bolsonaro_train, svc_X_bw_bolsonaro_test, svc_Y_bw_bolsonaro_train, svc_Y_bw_bolsonaro_test = train_test_split(X_bw_bolsonaro_train2, Y_bolsonaro_train, test_size=0.2)
svc_bolsonaro_bw_test = svc_bolsonaro_bw_test.fit(svc_X_bw_bolsonaro_train.toarray(), svc_Y_bw_bolsonaro_train)

score_svc_bw = svc_bolsonaro_bw_test.score(svc_X_bw_bolsonaro_test.toarray(), svc_Y_bw_bolsonaro_test)
svc_bolsonaro_bw = svc_bolsonaro_bw_test
for i in range(0,100):
    svc_bolsonaro_bw_test = LinearSVC()
    svc_X_bw_bolsonaro_train, svc_X_bw_bolsonaro_test, svc_Y_bw_bolsonaro_train, svc_Y_bw_bolsonaro_test = train_test_split(X_bw_bolsonaro_train2, Y_bolsonaro_train, test_size=0.2)
    svc_bolsonaro_bw_test = svc_bolsonaro_bw_test.fit(svc_X_bw_bolsonaro_train.toarray(), svc_Y_bw_bolsonaro_train)
    score_svc_bw_test = svc_bolsonaro_bw_test.score(svc_X_bw_bolsonaro_test.toarray(), svc_Y_bw_bolsonaro_test)
    if(score_svc_bw_test>score_svc_bw):
        score_svc_bw = score_svc_bw_test
        svc_bolsonaro_bw = svc_bolsonaro_bw_test
score_svc_bw

0.8079739625711961

In [48]:
i1=0
for i in range(5000,len(df_bolsonaro),5000):
    if (i==5000):
        predict_bolsonaro_bw_svc = svc_bolsonaro_bw.predict(Bag_of_words_teste(df_bolsonaro[i1:i], bw_bolsonaro).toarray())
    else:
        predict_bolsonaro_bw_svc = np.append(predict_bolsonaro_bw_svc, svc_bolsonaro_bw.predict(Bag_of_words_teste(df_bolsonaro[i1:i], bw_bolsonaro).toarray()))
    i1=i

predict_bolsonaro_bw_svc = np.append(predict_bolsonaro_bw_svc,svc_bolsonaro_bw.predict(Bag_of_words_teste(df_bolsonaro[i1:], bw_bolsonaro).toarray()))

df_bolsonaro['predict_bw_svc'] = predict_bolsonaro_bw_svc

np.unique(predict_bolsonaro_bw_svc, return_counts=True)

(array([-1,  0,  1]), array([125054,   4192, 817192], dtype=int64))

### Bolsonaro - SVM - tfidf

In [49]:
svc_bolsonaro_tfidf_test = LinearSVC()
svc_X_tfidf_bolsonaro_train, svc_X_tfidf_bolsonaro_test, svc_Y_tfidf_bolsonaro_train, svc_Y_tfidf_bolsonaro_test = train_test_split(X_tfidf_bolsonaro_train2, Y_bolsonaro_train, test_size=0.2)
svc_bolsonaro_tfidf_test = svc_bolsonaro_tfidf_test.fit(svc_X_tfidf_bolsonaro_train.toarray(), svc_Y_tfidf_bolsonaro_train)

score_svc_tfidf = svc_bolsonaro_tfidf_test.score(svc_X_tfidf_bolsonaro_test.toarray(), svc_Y_tfidf_bolsonaro_test)
svc_bolsonaro_tfidf = svc_bolsonaro_tfidf_test
for i in range(0,100):
    svc_bolsonaro_tfidf_test = LinearSVC()
    svc_X_tfidf_bolsonaro_train, svc_X_tfidf_bolsonaro_test, svc_Y_tfidf_bolsonaro_train, svc_Y_tfidf_bolsonaro_test = train_test_split(X_tfidf_bolsonaro_train2, Y_bolsonaro_train, test_size=0.2)
    svc_bolsonaro_tfidf_test = svc_bolsonaro_tfidf_test.fit(svc_X_tfidf_bolsonaro_train.toarray(), svc_Y_tfidf_bolsonaro_train)
    score_svc_tfidf_test = svc_bolsonaro_tfidf_test.score(svc_X_tfidf_bolsonaro_test.toarray(), svc_Y_tfidf_bolsonaro_test)
    if(score_svc_tfidf_test>score_svc_tfidf):
        score_svc_tfidf = score_svc_tfidf_test
        svc_bolsonaro_tfidf = svc_bolsonaro_tfidf_test
score_svc_tfidf

0.8543531326281529

In [50]:
i1=0
for i in range(5000,len(df_bolsonaro),5000):
    if (i==5000):
        predict_bolsonaro_tfidf_svc = svc_bolsonaro_tfidf.predict(tfidf_teste(df_bolsonaro[i1:i], tfidf_bolsonaro).toarray())
    else:
        predict_bolsonaro_tfidf_svc = np.append(predict_bolsonaro_tfidf_svc, svc_bolsonaro_tfidf.predict(tfidf_teste(df_bolsonaro[i1:i], tfidf_bolsonaro).toarray()))
    i1=i

predict_bolsonaro_tfidf_svc = np.append(predict_bolsonaro_tfidf_svc,svc_bolsonaro_tfidf.predict(tfidf_teste(df_bolsonaro[i1:], tfidf_bolsonaro).toarray()))

df_bolsonaro['predict_tfidf_svc'] = predict_bolsonaro_tfidf_svc

np.unique(predict_bolsonaro_tfidf_svc, return_counts=True)

(array([-1,  0,  1]), array([ 51153,    104, 895181], dtype=int64))

### Bolsonaro - MLP - bw

In [66]:
mlp_bolsonaro_bw_test = MLPClassifier()
mlp_X_bw_bolsonaro_train, mlp_X_bw_bolsonaro_test, mlp_Y_bw_bolsonaro_train, mlp_Y_bw_bolsonaro_test = train_test_split(X_bw_bolsonaro_train2, Y_bolsonaro_train, test_size=0.2)
mlp_bolsonaro_bw_test = mlp_bolsonaro_bw_test.fit(mlp_X_bw_bolsonaro_train.toarray(), mlp_Y_bw_bolsonaro_train)

score_mlp_bw = mlp_bolsonaro_bw_test.score(mlp_X_bw_bolsonaro_test.toarray(), mlp_Y_bw_bolsonaro_test)
mlp_bolsonaro_bw = mlp_bolsonaro_bw_test
for i in range(0,10):
    mlp_bolsonaro_bw_test = MLPClassifier()
    mlp_X_bw_bolsonaro_train, mlp_X_bw_bolsonaro_test, mlp_Y_bw_bolsonaro_train, mlp_Y_bw_bolsonaro_test = train_test_split(X_bw_bolsonaro_train2, Y_bolsonaro_train, test_size=0.2)
    mlp_bolsonaro_bw_test = mlp_bolsonaro_bw_test.fit(mlp_X_bw_bolsonaro_train.toarray(), mlp_Y_bw_bolsonaro_train)
    score_mlp_bw_test = mlp_bolsonaro_bw_test.score(mlp_X_bw_bolsonaro_test.toarray(), mlp_Y_bw_bolsonaro_test)
    if(score_mlp_bw_test>score_mlp_bw):
        score_mlp_bw = score_mlp_bw_test
        mlp_bolsonaro_bw = mlp_bolsonaro_bw_test
score_mlp_bw

0.7786818551668023

In [67]:
i1=0
for i in range(5000,len(df_bolsonaro),5000):
    if (i==5000):
        predict_bolsonaro_bw_mlp = mlp_bolsonaro_bw.predict(Bag_of_words_teste(df_bolsonaro[i1:i], bw_bolsonaro).toarray())
    else:
        predict_bolsonaro_bw_mlp = np.append(predict_bolsonaro_bw_mlp, mlp_bolsonaro_bw.predict(Bag_of_words_teste(df_bolsonaro[i1:i], bw_bolsonaro).toarray()))
    i1=i

predict_bolsonaro_bw_mlp = np.append(predict_bolsonaro_bw_mlp,mlp_bolsonaro_bw.predict(Bag_of_words_teste(df_bolsonaro[i1:], bw_bolsonaro).toarray()))

df_bolsonaro['predict_bw_mlp'] = predict_bolsonaro_bw_mlp

np.unique(predict_bolsonaro_bw_mlp, return_counts=True)

(array([-1,  0,  1]), array([151742,   4086, 790610], dtype=int64))

### Bolsonaro - MLP - tfidf

In [68]:
mlp_bolsonaro_tfidf_test = MLPClassifier()
mlp_X_tfidf_bolsonaro_train, mlp_X_tfidf_bolsonaro_test, mlp_Y_tfidf_bolsonaro_train, mlp_Y_tfidf_bolsonaro_test = train_test_split(X_tfidf_bolsonaro_train2, Y_bolsonaro_train, test_size=0.2)
mlp_bolsonaro_tfidf_test = mlp_bolsonaro_tfidf_test.fit(mlp_X_tfidf_bolsonaro_train.toarray(), mlp_Y_tfidf_bolsonaro_train)

score_mlp_tfidf = mlp_bolsonaro_tfidf_test.score(mlp_X_tfidf_bolsonaro_test.toarray(), mlp_Y_tfidf_bolsonaro_test)
mlp_bolsonaro_tfidf = mlp_bolsonaro_tfidf_test
for i in range(0,10):
    mlp_bolsonaro_tfidf_test = MLPClassifier()
    mlp_X_tfidf_bolsonaro_train, mlp_X_tfidf_bolsonaro_test, mlp_Y_tfidf_bolsonaro_train, mlp_Y_tfidf_bolsonaro_test = train_test_split(X_tfidf_bolsonaro_train2, Y_bolsonaro_train, test_size=0.2)
    mlp_bolsonaro_tfidf_test = mlp_bolsonaro_tfidf_test.fit(mlp_X_tfidf_bolsonaro_train.toarray(), mlp_Y_tfidf_bolsonaro_train)
    score_mlp_tfidf_test = mlp_bolsonaro_tfidf_test.score(mlp_X_tfidf_bolsonaro_test.toarray(), mlp_Y_tfidf_bolsonaro_test)
    if(score_mlp_tfidf_test>score_mlp_tfidf):
        score_mlp_tfidf = score_mlp_tfidf_test
        mlp_bolsonaro_tfidf = mlp_bolsonaro_tfidf_test
score_mlp_tfidf

0.774613506916192

In [69]:
i1=0
for i in range(5000,len(df_bolsonaro),5000):
    if (i==5000):
        predict_bolsonaro_tfidf_mlp = mlp_bolsonaro_tfidf.predict(tfidf_teste(df_bolsonaro[i1:i], tfidf_bolsonaro).toarray())
    else:
        predict_bolsonaro_tfidf_mlp = np.append(predict_bolsonaro_tfidf_mlp, mlp_bolsonaro_tfidf.predict(tfidf_teste(df_bolsonaro[i1:i], tfidf_bolsonaro).toarray()))
    i1=i

predict_bolsonaro_tfidf_mlp = np.append(predict_bolsonaro_tfidf_mlp,mlp_bolsonaro_tfidf.predict(tfidf_teste(df_bolsonaro[i1:], tfidf_bolsonaro).toarray()))

df_bolsonaro['predict_tfidf_mlp'] = predict_bolsonaro_tfidf_mlp

np.unique(predict_bolsonaro_tfidf_mlp, return_counts=True)

(array([-1,  0,  1]), array([144010,   1176, 801252], dtype=int64))

# Lula

### Lula - Arvore de Decisao - bw

In [70]:
X_bw_lula_train2 = X_bw_lula_train

tree_lula_bw_test = tree.DecisionTreeClassifier()
tree_X_bw_lula_train, tree_X_bw_lula_test, tree_Y_bw_lula_train, tree_Y_bw_lula_test = train_test_split(X_bw_lula_train2, Y_lula_train, test_size=0.2)
tree_lula_bw_test = tree_lula_bw_test.fit(tree_X_bw_lula_train, tree_Y_bw_lula_train)

score_tree_bw = tree_lula_bw_test.score(tree_X_bw_lula_test, tree_Y_bw_lula_test)
tree_lula_bw = tree_lula_bw_test
for i in range(0,100):
    tree_lula_bw_test = tree.DecisionTreeClassifier()
    tree_X_bw_lula_train, tree_X_bw_lula_test, tree_Y_bw_lula_train, tree_Y_bw_lula_test = train_test_split(X_bw_lula_train2, Y_lula_train, test_size=0.2)
    tree_lula_bw_test = tree_lula_bw_test.fit(tree_X_bw_lula_train, tree_Y_bw_lula_train)
    score_tree_bw_test = tree_lula_bw_test.score(tree_X_bw_lula_test, tree_Y_bw_lula_test)
    if(score_tree_bw_test>score_tree_bw):
        score_tree_bw = score_tree_bw_test
        tree_lula_bw = tree_lula_bw_test
score_tree_bw

0.8434531185460553

In [71]:
i1=0
for i in range(5000,len(df_lula),5000):
    if (i==5000):
        predict_lula_bw_tree = tree_lula_bw.predict(Bag_of_words_teste(df_lula[i1:i], bw_lula).toarray())
    else:
        predict_lula_bw_tree = np.append(predict_lula_bw_tree, tree_lula_bw.predict(Bag_of_words_teste(df_lula[i1:i], bw_lula).toarray()))
    i1=i

predict_lula_bw_tree = np.append(predict_lula_bw_tree,tree_lula_bw.predict(Bag_of_words_teste(df_lula[i1:], bw_lula).toarray()))


df_lula['predict_bw_ArvoreDeDecisao'] = predict_lula_bw_tree

np.unique(predict_lula_bw_tree, return_counts=True)

(array([-1,  0,  1]), array([ 97322,   1909, 832499], dtype=int64))

### Lula - Arvore de Decisao - tfidf

In [72]:
X_tfidf_lula_train2 = X_tfidf_lula_train

tree_lula_tfidf_test = tree.DecisionTreeClassifier()
tree_X_tfidf_lula_train, tree_X_tfidf_lula_test, tree_Y_tfidf_lula_train, tree_Y_tfidf_lula_test = train_test_split(X_tfidf_lula_train2, Y_lula_train, test_size=0.2)
tree_lula_tfidf_test = tree_lula_tfidf_test.fit(tree_X_tfidf_lula_train, tree_Y_tfidf_lula_train)

score_tree_tfidf = tree_lula_tfidf_test.score(tree_X_tfidf_lula_test, tree_Y_tfidf_lula_test)
tree_lula_tfidf = tree_lula_tfidf_test
for i in range(0,100):
    tree_lula_tfidf_test = tree.DecisionTreeClassifier()
    tree_X_tfidf_lula_train, tree_X_tfidf_lula_test, tree_Y_tfidf_lula_train, tree_Y_tfidf_lula_test = train_test_split(X_tfidf_lula_train2, Y_lula_train, test_size=0.2)
    tree_lula_tfidf_test = tree_lula_tfidf_test.fit(tree_X_tfidf_lula_train, tree_Y_tfidf_lula_train)
    score_tree_tfidf_test = tree_lula_tfidf_test.score(tree_X_tfidf_lula_test, tree_Y_tfidf_lula_test)
    if(score_tree_tfidf_test>score_tree_tfidf):
        score_tree_tfidf = score_tree_tfidf_test
        tree_lula_tfidf = tree_lula_tfidf_test
score_tree_tfidf

0.8405617513424205

In [73]:
i1=0
for i in range(5000,len(df_lula),5000):
    if (i==5000):
        predict_lula_tfidf_tree = tree_lula_tfidf.predict(tfidf_teste(df_lula[i1:i], tfidf_lula).toarray())
    else:
        predict_lula_tfidf_tree = np.append(predict_lula_tfidf_tree, tree_lula_tfidf.predict(tfidf_teste(df_lula[i1:i], tfidf_lula).toarray()))
    i1=i

predict_lula_tfidf_tree = np.append(predict_lula_tfidf_tree,tree_lula_tfidf.predict(tfidf_teste(df_lula[i1:], tfidf_lula).toarray()))


df_lula['predict_tfidf_ArvoreDeDecisao'] = predict_lula_tfidf_tree

np.unique(predict_lula_tfidf_tree, return_counts=True)

(array([-1,  0,  1]), array([134758,   1197, 795775], dtype=int64))

### Lula - Naive Bayes - bw

In [74]:
GaussianNB_lula_bw_test = GaussianNB()
GaussianNB_X_bw_lula_train, GaussianNB_X_bw_lula_test, GaussianNB_Y_bw_lula_train, GaussianNB_Y_bw_lula_test = train_test_split(X_bw_lula_train2, Y_lula_train, test_size=0.2)
GaussianNB_lula_bw_test = GaussianNB_lula_bw_test.fit(GaussianNB_X_bw_lula_train.toarray(), GaussianNB_Y_bw_lula_train)

score_GaussianNB_bw = GaussianNB_lula_bw_test.score(GaussianNB_X_bw_lula_test.toarray(), GaussianNB_Y_bw_lula_test)
GaussianNB_lula_bw = GaussianNB_lula_bw_test
for i in range(0,100):
    GaussianNB_lula_bw_test = GaussianNB()
    GaussianNB_X_bw_lula_train, GaussianNB_X_bw_lula_test, GaussianNB_Y_bw_lula_train, GaussianNB_Y_bw_lula_test = train_test_split(X_bw_lula_train2, Y_lula_train, test_size=0.2)
    GaussianNB_lula_bw_test = GaussianNB_lula_bw_test.fit(GaussianNB_X_bw_lula_train.toarray(), GaussianNB_Y_bw_lula_train)
    score_GaussianNB_bw_test = GaussianNB_lula_bw_test.score(GaussianNB_X_bw_lula_test.toarray(), GaussianNB_Y_bw_lula_test)
    if(score_GaussianNB_bw_test>score_GaussianNB_bw):
        score_GaussianNB_bw = score_GaussianNB_bw_test
        GaussianNB_lula_bw = GaussianNB_lula_bw_test
score_GaussianNB_bw

0.5192069392812887

In [75]:
i1=0
for i in range(5000,len(df_lula),5000):
    if (i==5000):
        predict_lula_bw_nb = GaussianNB_lula_bw.predict(Bag_of_words_teste(df_lula[i1:i], bw_lula).toarray())
    else:
        predict_lula_bw_nb = np.append(predict_lula_bw_nb, GaussianNB_lula_bw.predict(Bag_of_words_teste(df_lula[i1:i], bw_lula).toarray()))
    i1=i

predict_lula_bw_nb = np.append(predict_lula_bw_nb,GaussianNB_lula_bw.predict(Bag_of_words_teste(df_lula[i1:], bw_lula).toarray()))

df_lula['predict_bw_NaiveBayes'] = predict_lula_bw_nb

np.unique(predict_lula_bw_nb, return_counts=True)

(array([-1,  0,  1]), array([392813,   5483, 533434], dtype=int64))

### Lula - Naive Bayes - tfidf

In [76]:
GaussianNB_lula_tfidf_test = GaussianNB()
GaussianNB_X_tfidf_lula_train, GaussianNB_X_tfidf_lula_test, GaussianNB_Y_tfidf_lula_train, GaussianNB_Y_tfidf_lula_test = train_test_split(X_tfidf_lula_train2, Y_lula_train, test_size=0.2)
GaussianNB_lula_tfidf_test = GaussianNB_lula_tfidf_test.fit(GaussianNB_X_tfidf_lula_train.toarray(), GaussianNB_Y_tfidf_lula_train)

score_GaussianNB_tfidf = GaussianNB_lula_tfidf_test.score(GaussianNB_X_tfidf_lula_test.toarray(), GaussianNB_Y_tfidf_lula_test)
GaussianNB_lula_tfidf = GaussianNB_lula_tfidf_test
for i in range(0,100):
    GaussianNB_lula_tfidf_test = GaussianNB()
    GaussianNB_X_tfidf_lula_train, GaussianNB_X_tfidf_lula_test, GaussianNB_Y_tfidf_lula_train, GaussianNB_Y_tfidf_lula_test = train_test_split(X_tfidf_lula_train2, Y_lula_train, test_size=0.2)
    GaussianNB_lula_tfidf_test = GaussianNB_lula_tfidf_test.fit(GaussianNB_X_tfidf_lula_train.toarray(), GaussianNB_Y_tfidf_lula_train)
    score_GaussianNB_tfidf_test = GaussianNB_lula_tfidf_test.score(GaussianNB_X_tfidf_lula_test.toarray(), GaussianNB_Y_tfidf_lula_test)
    if(score_GaussianNB_tfidf_test>score_GaussianNB_tfidf):
        score_GaussianNB_tfidf = score_GaussianNB_tfidf_test
        GaussianNB_lula_tfidf = GaussianNB_lula_tfidf_test
score_GaussianNB_tfidf

0.5365551425030979

In [77]:
i1=0
for i in range(5000,len(df_lula),5000):
    if (i==5000):
        predict_lula_tfidf_nb = GaussianNB_lula_tfidf.predict(tfidf_teste(df_lula[i1:i], tfidf_lula).toarray())
    else:
        predict_lula_tfidf_nb = np.append(predict_lula_tfidf_nb, GaussianNB_lula_tfidf.predict(tfidf_teste(df_lula[i1:i], tfidf_lula).toarray()))
    i1=i

predict_lula_tfidf_nb = np.append(predict_lula_tfidf_nb,GaussianNB_lula_tfidf.predict(tfidf_teste(df_lula[i1:], tfidf_lula).toarray()))

df_lula['predict_tfidf_NaiveBayes'] = predict_lula_tfidf_nb

np.unique(predict_lula_tfidf_nb, return_counts=True)

(array([-1,  0,  1]), array([365118,   6326, 560286], dtype=int64))

### Lula - Random Forest - bw

In [78]:
RandomForest_lula_bw_test = RandomForestClassifier()
RandomForest_X_bw_lula_train, RandomForest_X_bw_lula_test, RandomForest_Y_bw_lula_train, RandomForest_Y_bw_lula_test = train_test_split(X_bw_lula_train2, Y_lula_train, test_size=0.2)
RandomForest_lula_bw_test = RandomForest_lula_bw_test.fit(RandomForest_X_bw_lula_train.toarray(), RandomForest_Y_bw_lula_train)

score_RandomForest_bw = RandomForest_lula_bw_test.score(RandomForest_X_bw_lula_test.toarray(), RandomForest_Y_bw_lula_test)
RandomForest_lula_bw = RandomForest_lula_bw_test
for i in range(0,100):
    RandomForest_lula_bw_test = RandomForestClassifier()
    RandomForest_X_bw_lula_train, RandomForest_X_bw_lula_test, RandomForest_Y_bw_lula_train, RandomForest_Y_bw_lula_test = train_test_split(X_bw_lula_train2, Y_lula_train, test_size=0.2)
    RandomForest_lula_bw_test = RandomForest_lula_bw_test.fit(RandomForest_X_bw_lula_train.toarray(), RandomForest_Y_bw_lula_train)
    score_RandomForest_bw_test = RandomForest_lula_bw_test.score(RandomForest_X_bw_lula_test.toarray(), RandomForest_Y_bw_lula_test)
    if(score_RandomForest_bw_test>score_RandomForest_bw):
        score_RandomForest_bw = score_RandomForest_bw_test
        RandomForest_lula_bw = RandomForest_lula_bw_test
score_RandomForest_bw

0.8806278397356464

In [79]:
i1=0
for i in range(5000,len(df_lula),5000):
    if (i==5000):
        predict_lula_bw_rf = RandomForest_lula_bw.predict(Bag_of_words_teste(df_lula[i1:i], bw_lula).toarray())
    else:
        predict_lula_bw_rf = np.append(predict_lula_bw_rf, RandomForest_lula_bw.predict(Bag_of_words_teste(df_lula[i1:i], bw_lula).toarray()))
    i1=i

predict_lula_bw_rf = np.append(predict_lula_bw_rf,RandomForest_lula_bw.predict(Bag_of_words_teste(df_lula[i1:], bw_lula).toarray()))

df_lula['predict_bw_RandomForest'] = predict_lula_bw_rf

np.unique(predict_lula_bw_rf, return_counts=True)

(array([-1,  0,  1]), array([ 18622,     26, 913082], dtype=int64))

### Lula - Random Forest - tfidf

In [80]:
RandomForest_lula_tfidf_test = RandomForestClassifier()
RandomForest_X_tfidf_lula_train, RandomForest_X_tfidf_lula_test, RandomForest_Y_tfidf_lula_train, RandomForest_Y_tfidf_lula_test = train_test_split(X_tfidf_lula_train2, Y_lula_train, test_size=0.2)
RandomForest_lula_tfidf_test = RandomForest_lula_tfidf_test.fit(RandomForest_X_tfidf_lula_train.toarray(), RandomForest_Y_tfidf_lula_train)

score_RandomForest_tfidf = RandomForest_lula_tfidf_test.score(RandomForest_X_tfidf_lula_test.toarray(), RandomForest_Y_tfidf_lula_test)
RandomForest_lula_tfidf = RandomForest_lula_tfidf_test
for i in range(0,100):
    RandomForest_lula_tfidf_test = RandomForestClassifier()
    RandomForest_X_tfidf_lula_train, RandomForest_X_tfidf_lula_test, RandomForest_Y_tfidf_lula_train, RandomForest_Y_tfidf_lula_test = train_test_split(X_tfidf_lula_train2, Y_lula_train, test_size=0.2)
    RandomForest_lula_tfidf_test = RandomForest_lula_tfidf_test.fit(RandomForest_X_tfidf_lula_train.toarray(), RandomForest_Y_tfidf_lula_train)
    score_RandomForest_tfidf_test = RandomForest_lula_tfidf_test.score(RandomForest_X_tfidf_lula_test.toarray(), RandomForest_Y_tfidf_lula_test)
    if(score_RandomForest_tfidf_test>score_RandomForest_tfidf):
        score_RandomForest_tfidf = score_RandomForest_tfidf_test
        RandomForest_lula_tfidf = RandomForest_lula_tfidf_test
score_RandomForest_tfidf

0.889301941346551

In [81]:
i1=0
for i in range(5000,len(df_lula),5000):
    if (i==5000):
        predict_lula_tfidf_rf = RandomForest_lula_tfidf.predict(tfidf_teste(df_lula[i1:i], tfidf_lula).toarray())
    else:
        predict_lula_tfidf_rf = np.append(predict_lula_tfidf_rf, RandomForest_lula_tfidf.predict(tfidf_teste(df_lula[i1:i], tfidf_lula).toarray()))
    i1=i

predict_lula_tfidf_rf = np.append(predict_lula_tfidf_rf,RandomForest_lula_tfidf.predict(tfidf_teste(df_lula[i1:], tfidf_lula).toarray()))

df_lula['predict_tfidf_RandomForest'] = predict_lula_tfidf_rf

np.unique(predict_lula_tfidf_rf, return_counts=True)

(array([-1,  0,  1]), array([ 20315,     25, 911390], dtype=int64))

### lula - SVM - bw

In [82]:
svc_lula_bw_test = LinearSVC()
svc_X_bw_lula_train, svc_X_bw_lula_test, svc_Y_bw_lula_train, svc_Y_bw_lula_test = train_test_split(X_bw_lula_train2, Y_lula_train, test_size=0.2)
svc_lula_bw_test = svc_lula_bw_test.fit(svc_X_bw_lula_train.toarray(), svc_Y_bw_lula_train)

score_svc_bw = svc_lula_bw_test.score(svc_X_bw_lula_test.toarray(), svc_Y_bw_lula_test)
svc_lula_bw = svc_lula_bw_test
for i in range(0,100):
    svc_lula_bw_test = LinearSVC()
    svc_X_bw_lula_train, svc_X_bw_lula_test, svc_Y_bw_lula_train, svc_Y_bw_lula_test = train_test_split(X_bw_lula_train2, Y_lula_train, test_size=0.2)
    svc_lula_bw_test = svc_lula_bw_test.fit(svc_X_bw_lula_train.toarray(), svc_Y_bw_lula_train)
    score_svc_bw_test = svc_lula_bw_test.score(svc_X_bw_lula_test.toarray(), svc_Y_bw_lula_test)
    if(score_svc_bw_test>score_svc_bw):
        score_svc_bw = score_svc_bw_test
        svc_lula_bw = svc_lula_bw_test
score_svc_bw

C:\Users\daian\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\daian\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\daian\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\daian\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\daian\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations

C:\Users\daian\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\daian\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\daian\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\daian\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\daian\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations

C:\Users\daian\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\daian\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\daian\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\daian\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\daian\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations

0.8451053283767038

In [83]:
i1=0
for i in range(5000,len(df_lula),5000):
    if (i==5000):
        predict_lula_bw_svc = svc_lula_bw.predict(Bag_of_words_teste(df_lula[i1:i], bw_lula).toarray())
    else:
        predict_lula_bw_svc = np.append(predict_lula_bw_svc, svc_lula_bw.predict(Bag_of_words_teste(df_lula[i1:i], bw_lula).toarray()))
    i1=i

predict_lula_bw_svc = np.append(predict_lula_bw_svc,svc_lula_bw.predict(Bag_of_words_teste(df_lula[i1:], bw_lula).toarray()))

df_lula['predict_bw_svc'] = predict_lula_bw_svc

np.unique(predict_lula_bw_svc, return_counts=True)

(array([-1,  0,  1]), array([126996,    127, 804607], dtype=int64))

### lula - SVM - tfidf

In [84]:
svc_lula_tfidf_test = LinearSVC()
svc_X_tfidf_lula_train, svc_X_tfidf_lula_test, svc_Y_tfidf_lula_train, svc_Y_tfidf_lula_test = train_test_split(X_tfidf_lula_train2, Y_lula_train, test_size=0.2)
svc_lula_tfidf_test = svc_lula_tfidf_test.fit(svc_X_tfidf_lula_train.toarray(), svc_Y_tfidf_lula_train)

score_svc_tfidf = svc_lula_tfidf_test.score(svc_X_tfidf_lula_test.toarray(), svc_Y_tfidf_lula_test)
svc_lula_tfidf = svc_lula_tfidf_test
for i in range(0,100):
    svc_lula_tfidf_test = LinearSVC()
    svc_X_tfidf_lula_train, svc_X_tfidf_lula_test, svc_Y_tfidf_lula_train, svc_Y_tfidf_lula_test = train_test_split(X_tfidf_lula_train2, Y_lula_train, test_size=0.2)
    svc_lula_tfidf_test = svc_lula_tfidf_test.fit(svc_X_tfidf_lula_train.toarray(), svc_Y_tfidf_lula_train)
    score_svc_tfidf_test = svc_lula_tfidf_test.score(svc_X_tfidf_lula_test.toarray(), svc_Y_tfidf_lula_test)
    if(score_svc_tfidf_test>score_svc_tfidf):
        score_svc_tfidf = score_svc_tfidf_test
        svc_lula_tfidf = svc_lula_tfidf_test
score_svc_tfidf

0.8793886823626601

In [85]:
i1=0
for i in range(5000,len(df_lula),5000):
    if (i==5000):
        predict_lula_tfidf_svc = svc_lula_tfidf.predict(tfidf_teste(df_lula[i1:i], tfidf_lula).toarray())
    else:
        predict_lula_tfidf_svc = np.append(predict_lula_tfidf_svc, svc_lula_tfidf.predict(tfidf_teste(df_lula[i1:i], tfidf_lula).toarray()))
    i1=i

predict_lula_tfidf_svc = np.append(predict_lula_tfidf_svc,svc_lula_tfidf.predict(tfidf_teste(df_lula[i1:], tfidf_lula).toarray()))

df_lula['predict_tfidf_svc'] = predict_lula_tfidf_svc

np.unique(predict_lula_tfidf_svc, return_counts=True)

(array([-1,  0,  1]), array([ 54700,     18, 877012], dtype=int64))

### Lula - MLP - bw

In [86]:
mlp_lula_bw_test = MLPClassifier()
mlp_X_bw_lula_train, mlp_X_bw_lula_test, mlp_Y_bw_lula_train, mlp_Y_bw_lula_test = train_test_split(X_bw_lula_train2, Y_lula_train, test_size=0.2)
mlp_lula_bw_test = mlp_lula_bw_test.fit(mlp_X_bw_lula_train.toarray(), mlp_Y_bw_lula_train)

score_mlp_bw = mlp_lula_bw_test.score(mlp_X_bw_lula_test.toarray(), mlp_Y_bw_lula_test)
mlp_lula_bw = mlp_lula_bw_test
for i in range(0,10):
    mlp_lula_bw_test = MLPClassifier()
    mlp_X_bw_lula_train, mlp_X_bw_lula_test, mlp_Y_bw_lula_train, mlp_Y_bw_lula_test = train_test_split(X_bw_lula_train2, Y_lula_train, test_size=0.2)
    mlp_lula_bw_test = mlp_lula_bw_test.fit(mlp_X_bw_lula_train.toarray(), mlp_Y_bw_lula_train)
    score_mlp_bw_test = mlp_lula_bw_test.score(mlp_X_bw_lula_test.toarray(), mlp_Y_bw_lula_test)
    if(score_mlp_bw_test>score_mlp_bw):
        score_mlp_bw = score_mlp_bw_test
        mlp_lula_bw = mlp_lula_bw_test
score_mlp_bw

0.8327137546468402

In [87]:
i1=0
for i in range(5000,len(df_lula),5000):
    if (i==5000):
        predict_lula_bw_mlp = mlp_lula_bw.predict(Bag_of_words_teste(df_lula[i1:i], bw_lula).toarray())
    else:
        predict_lula_bw_mlp = np.append(predict_lula_bw_mlp, mlp_lula_bw.predict(Bag_of_words_teste(df_lula[i1:i], bw_lula).toarray()))
    i1=i

predict_lula_bw_mlp = np.append(predict_lula_bw_mlp,mlp_lula_bw.predict(Bag_of_words_teste(df_lula[i1:], bw_lula).toarray()))

df_lula['predict_bw_mlp'] = predict_lula_bw_mlp

np.unique(predict_lula_bw_mlp, return_counts=True)

(array([-1,  0,  1]), array([103552,    311, 827867], dtype=int64))

### Lula - MLP - tfidf

In [88]:
mlp_lula_tfidf_test = MLPClassifier()
mlp_X_tfidf_lula_train, mlp_X_tfidf_lula_test, mlp_Y_tfidf_lula_train, mlp_Y_tfidf_lula_test = train_test_split(X_tfidf_lula_train2, Y_lula_train, test_size=0.2)
mlp_lula_tfidf_test = mlp_lula_tfidf_test.fit(mlp_X_tfidf_lula_train.toarray(), mlp_Y_tfidf_lula_train)

score_mlp_tfidf = mlp_lula_tfidf_test.score(mlp_X_tfidf_lula_test.toarray(), mlp_Y_tfidf_lula_test)
mlp_lula_tfidf = mlp_lula_tfidf_test
for i in range(0,10):
    mlp_lula_tfidf_test = MLPClassifier()
    mlp_X_tfidf_lula_train, mlp_X_tfidf_lula_test, mlp_Y_tfidf_lula_train, mlp_Y_tfidf_lula_test = train_test_split(X_tfidf_lula_train2, Y_lula_train, test_size=0.2)
    mlp_lula_tfidf_test = mlp_lula_tfidf_test.fit(mlp_X_tfidf_lula_train.toarray(), mlp_Y_tfidf_lula_train)
    score_mlp_tfidf_test = mlp_lula_tfidf_test.score(mlp_X_tfidf_lula_test.toarray(), mlp_Y_tfidf_lula_test)
    if(score_mlp_tfidf_test>score_mlp_tfidf):
        score_mlp_tfidf = score_mlp_tfidf_test
        mlp_lula_tfidf = mlp_lula_tfidf_test
score_mlp_tfidf

0.8446922759190417

In [89]:
i1=0
for i in range(5000,len(df_lula),5000):
    if (i==5000):
        predict_lula_tfidf_mlp = mlp_lula_tfidf.predict(tfidf_teste(df_lula[i1:i], tfidf_lula).toarray())
    else:
        predict_lula_tfidf_mlp = np.append(predict_lula_tfidf_mlp, mlp_lula_tfidf.predict(tfidf_teste(df_lula[i1:i], tfidf_lula).toarray()))
    i1=i

predict_lula_tfidf_mlp = np.append(predict_lula_tfidf_mlp,mlp_lula_tfidf.predict(tfidf_teste(df_lula[i1:], tfidf_lula).toarray()))

df_lula['predict_tfidf_mlp'] = predict_lula_tfidf_mlp

np.unique(predict_lula_tfidf_mlp, return_counts=True)

(array([-1,  0,  1]), array([117362,    141, 814227], dtype=int64))

# Simone

### Simone - Arvore de Decisao - bw

In [90]:
X_bw_simone_train2 = X_bw_simone_train

tree_simone_bw_test = tree.DecisionTreeClassifier()
tree_X_bw_simone_train, tree_X_bw_simone_test, tree_Y_bw_simone_train, tree_Y_bw_simone_test = train_test_split(X_bw_simone_train2, Y_simone_train, test_size=0.2)
tree_simone_bw_test = tree_simone_bw_test.fit(tree_X_bw_simone_train, tree_Y_bw_simone_train)

score_tree_bw = tree_simone_bw_test.score(tree_X_bw_simone_test, tree_Y_bw_simone_test)
tree_simone_bw = tree_simone_bw_test
for i in range(0,100):
    tree_simone_bw_test = tree.DecisionTreeClassifier()
    tree_X_bw_simone_train, tree_X_bw_simone_test, tree_Y_bw_simone_train, tree_Y_bw_simone_test = train_test_split(X_bw_simone_train2, Y_simone_train, test_size=0.2)
    tree_simone_bw_test = tree_simone_bw_test.fit(tree_X_bw_simone_train, tree_Y_bw_simone_train)
    score_tree_bw_test = tree_simone_bw_test.score(tree_X_bw_simone_test, tree_Y_bw_simone_test)
    if(score_tree_bw_test>score_tree_bw):
        score_tree_bw = score_tree_bw_test
        tree_simone_bw = tree_simone_bw_test
score_tree_bw

0.7083333333333334

In [91]:
i1=0
for i in range(5000,len(df_simone),5000):
    if (i==5000):
        predict_simone_bw_tree = tree_simone_bw.predict(Bag_of_words_teste(df_simone[i1:i], bw_simone).toarray())
    else:
        predict_simone_bw_tree = np.append(predict_simone_bw_tree, tree_simone_bw.predict(Bag_of_words_teste(df_simone[i1:i], bw_simone).toarray()))
    i1=i

predict_simone_bw_tree = np.append(predict_simone_bw_tree,tree_simone_bw.predict(Bag_of_words_teste(df_simone[i1:], bw_simone).toarray()))


df_simone['predict_bw_ArvoreDeDecisao'] = predict_simone_bw_tree

np.unique(predict_simone_bw_tree, return_counts=True)

(array([-1,  0,  1]), array([ 7945, 78065, 45152], dtype=int64))

### Simone - Arvore de Decisao - tfidf

In [95]:
X_tfidf_simone_train2 = X_tfidf_simone_train

tree_simone_tfidf_test = tree.DecisionTreeClassifier()
tree_X_tfidf_simone_train, tree_X_tfidf_simone_test, tree_Y_tfidf_simone_train, tree_Y_tfidf_simone_test = train_test_split(X_tfidf_simone_train2, Y_simone_train, test_size=0.2)
tree_simone_tfidf_test = tree_simone_tfidf_test.fit(tree_X_tfidf_simone_train, tree_Y_tfidf_simone_train)

score_tree_tfidf = tree_simone_tfidf_test.score(tree_X_tfidf_simone_test, tree_Y_tfidf_simone_test)
tree_simone_tfidf = tree_simone_tfidf_test
for i in range(0,100):
    tree_simone_tfidf_test = tree.DecisionTreeClassifier()
    tree_X_tfidf_simone_train, tree_X_tfidf_simone_test, tree_Y_tfidf_simone_train, tree_Y_tfidf_simone_test = train_test_split(X_tfidf_simone_train2, Y_simone_train, test_size=0.2)
    tree_simone_tfidf_test = tree_simone_tfidf_test.fit(tree_X_tfidf_simone_train, tree_Y_tfidf_simone_train)
    score_tree_tfidf_test = tree_simone_tfidf_test.score(tree_X_tfidf_simone_test, tree_Y_tfidf_simone_test)
    if(score_tree_tfidf_test>score_tree_tfidf):
        score_tree_tfidf = score_tree_tfidf_test
        tree_simone_tfidf = tree_simone_tfidf_test
score_tree_tfidf

0.7708333333333334

In [96]:
i1=0
for i in range(5000,len(df_simone),5000):
    if (i==5000):
        predict_simone_tfidf_tree = tree_simone_tfidf.predict(tfidf_teste(df_simone[i1:i], tfidf_simone).toarray())
    else:
        predict_simone_tfidf_tree = np.append(predict_simone_tfidf_tree, tree_simone_tfidf.predict(tfidf_teste(df_simone[i1:i], tfidf_simone).toarray()))
    i1=i

predict_simone_tfidf_tree = np.append(predict_simone_tfidf_tree,tree_simone_tfidf.predict(tfidf_teste(df_simone[i1:], tfidf_simone).toarray()))


df_simone['predict_tfidf_ArvoreDeDecisao'] = predict_simone_tfidf_tree

np.unique(predict_simone_tfidf_tree, return_counts=True)

(array([-1,  0,  1]), array([ 5426, 66730, 59006], dtype=int64))

### Simone - Naive Bayes - bw

In [98]:
GaussianNB_simone_bw_test = GaussianNB()
GaussianNB_X_bw_simone_train, GaussianNB_X_bw_simone_test, GaussianNB_Y_bw_simone_train, GaussianNB_Y_bw_simone_test = train_test_split(X_bw_simone_train2, Y_simone_train, test_size=0.2)
GaussianNB_simone_bw_test = GaussianNB_simone_bw_test.fit(GaussianNB_X_bw_simone_train.toarray(), GaussianNB_Y_bw_simone_train)

score_GaussianNB_bw = GaussianNB_simone_bw_test.score(GaussianNB_X_bw_simone_test.toarray(), GaussianNB_Y_bw_simone_test)
GaussianNB_simone_bw = GaussianNB_simone_bw_test
for i in range(0,100):
    GaussianNB_simone_bw_test = GaussianNB()
    GaussianNB_X_bw_simone_train, GaussianNB_X_bw_simone_test, GaussianNB_Y_bw_simone_train, GaussianNB_Y_bw_simone_test = train_test_split(X_bw_simone_train2, Y_simone_train, test_size=0.2)
    GaussianNB_simone_bw_test = GaussianNB_simone_bw_test.fit(GaussianNB_X_bw_simone_train.toarray(), GaussianNB_Y_bw_simone_train)
    score_GaussianNB_bw_test = GaussianNB_simone_bw_test.score(GaussianNB_X_bw_simone_test.toarray(), GaussianNB_Y_bw_simone_test)
    if(score_GaussianNB_bw_test>score_GaussianNB_bw):
        score_GaussianNB_bw = score_GaussianNB_bw_test
        GaussianNB_simone_bw = GaussianNB_simone_bw_test
score_GaussianNB_bw

0.7291666666666666

In [99]:
i1=0
for i in range(5000,len(df_simone),5000):
    if (i==5000):
        predict_simone_bw_nb = GaussianNB_simone_bw.predict(Bag_of_words_teste(df_simone[i1:i], bw_simone).toarray())
    else:
        predict_simone_bw_nb = np.append(predict_simone_bw_nb, GaussianNB_simone_bw.predict(Bag_of_words_teste(df_simone[i1:i], bw_simone).toarray()))
    i1=i

predict_simone_bw_nb = np.append(predict_simone_bw_nb,GaussianNB_simone_bw.predict(Bag_of_words_teste(df_simone[i1:], bw_simone).toarray()))

df_simone['predict_bw_NaiveBayes'] = predict_simone_bw_nb

np.unique(predict_simone_bw_nb, return_counts=True)

(array([-1,  0,  1]), array([11932, 68318, 50912], dtype=int64))

### Simone - Naive Bayes - tfidf

In [100]:
GaussianNB_simone_tfidf_test = GaussianNB()
GaussianNB_X_tfidf_simone_train, GaussianNB_X_tfidf_simone_test, GaussianNB_Y_tfidf_simone_train, GaussianNB_Y_tfidf_simone_test = train_test_split(X_tfidf_simone_train2, Y_simone_train, test_size=0.2)
GaussianNB_simone_tfidf_test = GaussianNB_simone_tfidf_test.fit(GaussianNB_X_tfidf_simone_train.toarray(), GaussianNB_Y_tfidf_simone_train)

score_GaussianNB_tfidf = GaussianNB_simone_tfidf_test.score(GaussianNB_X_tfidf_simone_test.toarray(), GaussianNB_Y_tfidf_simone_test)
GaussianNB_simone_tfidf = GaussianNB_simone_tfidf_test
for i in range(0,100):
    GaussianNB_simone_tfidf_test = GaussianNB()
    GaussianNB_X_tfidf_simone_train, GaussianNB_X_tfidf_simone_test, GaussianNB_Y_tfidf_simone_train, GaussianNB_Y_tfidf_simone_test = train_test_split(X_tfidf_simone_train2, Y_simone_train, test_size=0.2)
    GaussianNB_simone_tfidf_test = GaussianNB_simone_tfidf_test.fit(GaussianNB_X_tfidf_simone_train.toarray(), GaussianNB_Y_tfidf_simone_train)
    score_GaussianNB_tfidf_test = GaussianNB_simone_tfidf_test.score(GaussianNB_X_tfidf_simone_test.toarray(), GaussianNB_Y_tfidf_simone_test)
    if(score_GaussianNB_tfidf_test>score_GaussianNB_tfidf):
        score_GaussianNB_tfidf = score_GaussianNB_tfidf_test
        GaussianNB_simone_tfidf = GaussianNB_simone_tfidf_test
score_GaussianNB_tfidf

0.75

In [101]:
i1=0
for i in range(5000,len(df_simone),5000):
    if (i==5000):
        predict_simone_tfidf_nb = GaussianNB_simone_tfidf.predict(tfidf_teste(df_simone[i1:i], tfidf_simone).toarray())
    else:
        predict_simone_tfidf_nb = np.append(predict_simone_tfidf_nb, GaussianNB_simone_tfidf.predict(tfidf_teste(df_simone[i1:i], tfidf_simone).toarray()))
    i1=i

predict_simone_tfidf_nb = np.append(predict_simone_tfidf_nb,GaussianNB_simone_tfidf.predict(tfidf_teste(df_simone[i1:], tfidf_simone).toarray()))

df_simone['predict_tfidf_NaiveBayes'] = predict_simone_tfidf_nb

np.unique(predict_simone_tfidf_nb, return_counts=True)

(array([-1,  0,  1]), array([13229, 55522, 62411], dtype=int64))

### Simone - Random Forest - bw

In [102]:
RandomForest_simone_bw_test = RandomForestClassifier()
RandomForest_X_bw_simone_train, RandomForest_X_bw_simone_test, RandomForest_Y_bw_simone_train, RandomForest_Y_bw_simone_test = train_test_split(X_bw_simone_train2, Y_simone_train, test_size=0.2)
RandomForest_simone_bw_test = RandomForest_simone_bw_test.fit(RandomForest_X_bw_simone_train.toarray(), RandomForest_Y_bw_simone_train)

score_RandomForest_bw = RandomForest_simone_bw_test.score(RandomForest_X_bw_simone_test.toarray(), RandomForest_Y_bw_simone_test)
RandomForest_simone_bw = RandomForest_simone_bw_test
for i in range(0,100):
    RandomForest_simone_bw_test = RandomForestClassifier()
    RandomForest_X_bw_simone_train, RandomForest_X_bw_simone_test, RandomForest_Y_bw_simone_train, RandomForest_Y_bw_simone_test = train_test_split(X_bw_simone_train2, Y_simone_train, test_size=0.2)
    RandomForest_simone_bw_test = RandomForest_simone_bw_test.fit(RandomForest_X_bw_simone_train.toarray(), RandomForest_Y_bw_simone_train)
    score_RandomForest_bw_test = RandomForest_simone_bw_test.score(RandomForest_X_bw_simone_test.toarray(), RandomForest_Y_bw_simone_test)
    if(score_RandomForest_bw_test>score_RandomForest_bw):
        score_RandomForest_bw = score_RandomForest_bw_test
        RandomForest_simone_bw = RandomForest_simone_bw_test
score_RandomForest_bw

0.7708333333333334

In [103]:
i1=0
for i in range(5000,len(df_simone),5000):
    if (i==5000):
        predict_simone_bw_rf = RandomForest_simone_bw.predict(Bag_of_words_teste(df_simone[i1:i], bw_simone).toarray())
    else:
        predict_simone_bw_rf = np.append(predict_simone_bw_rf, RandomForest_simone_bw.predict(Bag_of_words_teste(df_simone[i1:i], bw_simone).toarray()))
    i1=i

predict_simone_bw_rf = np.append(predict_simone_bw_rf,RandomForest_simone_bw.predict(Bag_of_words_teste(df_simone[i1:], bw_simone).toarray()))

df_simone['predict_bw_RandomForest'] = predict_simone_bw_rf

np.unique(predict_simone_bw_rf, return_counts=True)

(array([-1,  0,  1]), array([  102, 43078, 87982], dtype=int64))

### Simone - Random Forest - tfidf

In [104]:
RandomForest_simone_tfidf_test = RandomForestClassifier()
RandomForest_X_tfidf_simone_train, RandomForest_X_tfidf_simone_test, RandomForest_Y_tfidf_simone_train, RandomForest_Y_tfidf_simone_test = train_test_split(X_tfidf_simone_train2, Y_simone_train, test_size=0.2)
RandomForest_simone_tfidf_test = RandomForest_simone_tfidf_test.fit(RandomForest_X_tfidf_simone_train.toarray(), RandomForest_Y_tfidf_simone_train)

score_RandomForest_tfidf = RandomForest_simone_tfidf_test.score(RandomForest_X_tfidf_simone_test.toarray(), RandomForest_Y_tfidf_simone_test)
RandomForest_simone_tfidf = RandomForest_simone_tfidf_test
for i in range(0,100):
    RandomForest_simone_tfidf_test = RandomForestClassifier()
    RandomForest_X_tfidf_simone_train, RandomForest_X_tfidf_simone_test, RandomForest_Y_tfidf_simone_train, RandomForest_Y_tfidf_simone_test = train_test_split(X_tfidf_simone_train2, Y_simone_train, test_size=0.2)
    RandomForest_simone_tfidf_test = RandomForest_simone_tfidf_test.fit(RandomForest_X_tfidf_simone_train.toarray(), RandomForest_Y_tfidf_simone_train)
    score_RandomForest_tfidf_test = RandomForest_simone_tfidf_test.score(RandomForest_X_tfidf_simone_test.toarray(), RandomForest_Y_tfidf_simone_test)
    if(score_RandomForest_tfidf_test>score_RandomForest_tfidf):
        score_RandomForest_tfidf = score_RandomForest_tfidf_test
        RandomForest_simone_tfidf = RandomForest_simone_tfidf_test
score_RandomForest_tfidf

0.8333333333333334

In [105]:
i1=0
for i in range(5000,len(df_simone),5000):
    if (i==5000):
        predict_simone_tfidf_rf = RandomForest_simone_tfidf.predict(tfidf_teste(df_simone[i1:i], tfidf_simone).toarray())
    else:
        predict_simone_tfidf_rf = np.append(predict_simone_tfidf_rf, RandomForest_simone_tfidf.predict(tfidf_teste(df_simone[i1:i], tfidf_simone).toarray()))
    i1=i

predict_simone_tfidf_rf = np.append(predict_simone_tfidf_rf,RandomForest_simone_tfidf.predict(tfidf_teste(df_simone[i1:], tfidf_simone).toarray()))

df_simone['predict_tfidf_RandomForest'] = predict_simone_tfidf_rf

np.unique(predict_simone_tfidf_rf, return_counts=True)

(array([-1,  0,  1]), array([  111, 59182, 71869], dtype=int64))

### Simone - SVM - bw

In [106]:
svc_simone_bw_test = LinearSVC()
svc_X_bw_simone_train, svc_X_bw_simone_test, svc_Y_bw_simone_train, svc_Y_bw_simone_test = train_test_split(X_bw_simone_train2, Y_simone_train, test_size=0.2)
svc_simone_bw_test = svc_simone_bw_test.fit(svc_X_bw_simone_train.toarray(), svc_Y_bw_simone_train)

score_svc_bw = svc_simone_bw_test.score(svc_X_bw_simone_test.toarray(), svc_Y_bw_simone_test)
svc_simone_bw = svc_simone_bw_test
for i in range(0,100):
    svc_simone_bw_test = LinearSVC()
    svc_X_bw_simone_train, svc_X_bw_simone_test, svc_Y_bw_simone_train, svc_Y_bw_simone_test = train_test_split(X_bw_simone_train2, Y_simone_train, test_size=0.2)
    svc_simone_bw_test = svc_simone_bw_test.fit(svc_X_bw_simone_train.toarray(), svc_Y_bw_simone_train)
    score_svc_bw_test = svc_simone_bw_test.score(svc_X_bw_simone_test.toarray(), svc_Y_bw_simone_test)
    if(score_svc_bw_test>score_svc_bw):
        score_svc_bw = score_svc_bw_test
        svc_simone_bw = svc_simone_bw_test
score_svc_bw

0.8125

In [107]:
i1=0
for i in range(5000,len(df_simone),5000):
    if (i==5000):
        predict_simone_bw_svc = svc_simone_bw.predict(Bag_of_words_teste(df_simone[i1:i], bw_simone).toarray())
    else:
        predict_simone_bw_svc = np.append(predict_simone_bw_svc, svc_simone_bw.predict(Bag_of_words_teste(df_simone[i1:i], bw_simone).toarray()))
    i1=i

predict_simone_bw_svc = np.append(predict_simone_bw_svc,svc_simone_bw.predict(Bag_of_words_teste(df_simone[i1:], bw_simone).toarray()))

df_simone['predict_bw_svc'] = predict_simone_bw_svc

np.unique(predict_simone_bw_svc, return_counts=True)

(array([-1,  0,  1]), array([ 6125, 62159, 62878], dtype=int64))

### Simone - SVM - tfidf

In [108]:
svc_simone_tfidf_test = LinearSVC()
svc_X_tfidf_simone_train, svc_X_tfidf_simone_test, svc_Y_tfidf_simone_train, svc_Y_tfidf_simone_test = train_test_split(X_tfidf_simone_train2, Y_simone_train, test_size=0.2)
svc_simone_tfidf_test = svc_simone_tfidf_test.fit(svc_X_tfidf_simone_train.toarray(), svc_Y_tfidf_simone_train)

score_svc_tfidf = svc_simone_tfidf_test.score(svc_X_tfidf_simone_test.toarray(), svc_Y_tfidf_simone_test)
svc_simone_tfidf = svc_simone_tfidf_test
for i in range(0,100):
    svc_simone_tfidf_test = LinearSVC()
    svc_X_tfidf_simone_train, svc_X_tfidf_simone_test, svc_Y_tfidf_simone_train, svc_Y_tfidf_simone_test = train_test_split(X_tfidf_simone_train2, Y_simone_train, test_size=0.2)
    svc_simone_tfidf_test = svc_simone_tfidf_test.fit(svc_X_tfidf_simone_train.toarray(), svc_Y_tfidf_simone_train)
    score_svc_tfidf_test = svc_simone_tfidf_test.score(svc_X_tfidf_simone_test.toarray(), svc_Y_tfidf_simone_test)
    if(score_svc_tfidf_test>score_svc_tfidf):
        score_svc_tfidf = score_svc_tfidf_test
        svc_simone_tfidf = svc_simone_tfidf_test
score_svc_tfidf

0.8333333333333334

In [109]:
i1=0
for i in range(5000,len(df_simone),5000):
    if (i==5000):
        predict_simone_tfidf_svc = svc_simone_tfidf.predict(tfidf_teste(df_simone[i1:i], tfidf_simone).toarray())
    else:
        predict_simone_tfidf_svc = np.append(predict_simone_tfidf_svc, svc_simone_tfidf.predict(tfidf_teste(df_simone[i1:i], tfidf_simone).toarray()))
    i1=i

predict_simone_tfidf_svc = np.append(predict_simone_tfidf_svc,svc_simone_tfidf.predict(tfidf_teste(df_simone[i1:], tfidf_simone).toarray()))

df_simone['predict_tfidf_svc'] = predict_simone_tfidf_svc

np.unique(predict_simone_tfidf_svc, return_counts=True)

(array([-1,  0,  1]), array([ 9325, 59014, 62823], dtype=int64))

### Simone - MLP - bw

In [110]:
mlp_simone_bw_test = MLPClassifier()
mlp_X_bw_simone_train, mlp_X_bw_simone_test, mlp_Y_bw_simone_train, mlp_Y_bw_simone_test = train_test_split(X_bw_simone_train2, Y_simone_train, test_size=0.2)
mlp_simone_bw_test = mlp_simone_bw_test.fit(mlp_X_bw_simone_train.toarray(), mlp_Y_bw_simone_train)

score_mlp_bw = mlp_simone_bw_test.score(mlp_X_bw_simone_test.toarray(), mlp_Y_bw_simone_test)
mlp_simone_bw = mlp_simone_bw_test
for i in range(0,10):
    mlp_simone_bw_test = MLPClassifier()
    mlp_X_bw_simone_train, mlp_X_bw_simone_test, mlp_Y_bw_simone_train, mlp_Y_bw_simone_test = train_test_split(X_bw_simone_train2, Y_simone_train, test_size=0.2)
    mlp_simone_bw_test = mlp_simone_bw_test.fit(mlp_X_bw_simone_train.toarray(), mlp_Y_bw_simone_train)
    score_mlp_bw_test = mlp_simone_bw_test.score(mlp_X_bw_simone_test.toarray(), mlp_Y_bw_simone_test)
    if(score_mlp_bw_test>score_mlp_bw):
        score_mlp_bw = score_mlp_bw_test
        mlp_simone_bw = mlp_simone_bw_test
score_mlp_bw

C:\Users\daian\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\daian\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\daian\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\daian\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\daian\anaconda3\lib\site-packages\sklearn\neural_network\_multi

0.75

In [111]:
i1=0
for i in range(5000,len(df_simone),5000):
    if (i==5000):
        predict_simone_bw_mlp = mlp_simone_bw.predict(Bag_of_words_teste(df_simone[i1:i], bw_simone).toarray())
    else:
        predict_simone_bw_mlp = np.append(predict_simone_bw_mlp, mlp_simone_bw.predict(Bag_of_words_teste(df_simone[i1:i], bw_simone).toarray()))
    i1=i

predict_simone_bw_mlp = np.append(predict_simone_bw_mlp,mlp_simone_bw.predict(Bag_of_words_teste(df_simone[i1:], bw_simone).toarray()))

df_simone['predict_bw_mlp'] = predict_simone_bw_mlp
df_simone['predict_bw_mlp2'] = predict_simone_bw_mlp

np.unique(predict_simone_bw_mlp, return_counts=True)

(array([-1,  0,  1]), array([ 2388, 65680, 63094], dtype=int64))

### Simone - MLP - tfidf

In [112]:
mlp_simone_tfidf_test = MLPClassifier()
mlp_X_tfidf_simone_train, mlp_X_tfidf_simone_test, mlp_Y_tfidf_simone_train, mlp_Y_tfidf_simone_test = train_test_split(X_tfidf_simone_train2, Y_simone_train, test_size=0.2)
mlp_simone_tfidf_test = mlp_simone_tfidf_test.fit(mlp_X_tfidf_simone_train.toarray(), mlp_Y_tfidf_simone_train)

score_mlp_tfidf = mlp_simone_tfidf_test.score(mlp_X_tfidf_simone_test.toarray(), mlp_Y_tfidf_simone_test)
mlp_simone_tfidf = mlp_simone_tfidf_test
for i in range(0,10):
    mlp_simone_tfidf_test = MLPClassifier()
    mlp_X_tfidf_simone_train, mlp_X_tfidf_simone_test, mlp_Y_tfidf_simone_train, mlp_Y_tfidf_simone_test = train_test_split(X_tfidf_simone_train2, Y_simone_train, test_size=0.2)
    mlp_simone_tfidf_test = mlp_simone_tfidf_test.fit(mlp_X_tfidf_simone_train.toarray(), mlp_Y_tfidf_simone_train)
    score_mlp_tfidf_test = mlp_simone_tfidf_test.score(mlp_X_tfidf_simone_test.toarray(), mlp_Y_tfidf_simone_test)
    if(score_mlp_tfidf_test>score_mlp_tfidf):
        score_mlp_tfidf = score_mlp_tfidf_test
        mlp_simone_tfidf = mlp_simone_tfidf_test
score_mlp_tfidf

C:\Users\daian\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\daian\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\daian\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\daian\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\daian\anaconda3\lib\site-packages\sklearn\neural_network\_multi

0.7083333333333334

In [113]:
i1=0
for i in range(5000,len(df_simone),5000):
    if (i==5000):
        predict_simone_tfidf_mlp = mlp_simone_tfidf.predict(tfidf_teste(df_simone[i1:i], tfidf_simone).toarray())
    else:
        predict_simone_tfidf_mlp = np.append(predict_simone_tfidf_mlp, mlp_simone_tfidf.predict(tfidf_teste(df_simone[i1:i], tfidf_simone).toarray()))
    i1=i

predict_simone_tfidf_mlp = np.append(predict_simone_tfidf_mlp,mlp_simone_tfidf.predict(tfidf_teste(df_simone[i1:], tfidf_simone).toarray()))

df_simone['predict_tfidf_mlp'] = predict_simone_tfidf_mlp
df_simone['predict_tfidf_mlp2'] = predict_simone_tfidf_mlp

np.unique(predict_simone_tfidf_mlp, return_counts=True)

(array([-1,  0,  1]), array([ 9972, 52189, 69001], dtype=int64))

# Ciro

### Ciro - Arvore de Decisao - bw

In [114]:
X_bw_ciro_train2 = X_bw_ciro_train

tree_ciro_bw_test = tree.DecisionTreeClassifier()
tree_X_bw_ciro_train, tree_X_bw_ciro_test, tree_Y_bw_ciro_train, tree_Y_bw_ciro_test = train_test_split(X_bw_ciro_train2, Y_ciro_train, test_size=0.2)
tree_ciro_bw_test = tree_ciro_bw_test.fit(tree_X_bw_ciro_train, tree_Y_bw_ciro_train)

score_tree_bw = tree_ciro_bw_test.score(tree_X_bw_ciro_test, tree_Y_bw_ciro_test)
tree_ciro_bw = tree_ciro_bw_test
for i in range(0,100):
    tree_ciro_bw_test = tree.DecisionTreeClassifier()
    tree_X_bw_ciro_train, tree_X_bw_ciro_test, tree_Y_bw_ciro_train, tree_Y_bw_ciro_test = train_test_split(X_bw_ciro_train2, Y_ciro_train, test_size=0.2)
    tree_ciro_bw_test = tree_ciro_bw_test.fit(tree_X_bw_ciro_train, tree_Y_bw_ciro_train)
    score_tree_bw_test = tree_ciro_bw_test.score(tree_X_bw_ciro_test, tree_Y_bw_ciro_test)
    if(score_tree_bw_test>score_tree_bw):
        score_tree_bw = score_tree_bw_test
        tree_ciro_bw = tree_ciro_bw_test
score_tree_bw

0.9992636229749632

In [115]:
i1=0
for i in range(5000,len(df_ciro),5000):
    if (i==5000):
        predict_ciro_bw_tree = tree_ciro_bw.predict(Bag_of_words_teste(df_ciro[i1:i], bw_ciro).toarray())
    else:
        predict_ciro_bw_tree = np.append(predict_ciro_bw_tree, tree_ciro_bw.predict(Bag_of_words_teste(df_ciro[i1:i], bw_ciro).toarray()))
    i1=i

predict_ciro_bw_tree = np.append(predict_ciro_bw_tree,tree_ciro_bw.predict(Bag_of_words_teste(df_ciro[i1:], bw_ciro).toarray()))


df_ciro['predict_bw_ArvoreDeDecisao'] = predict_ciro_bw_tree

np.unique(predict_ciro_bw_tree, return_counts=True)

(array([-1,  0,  1]), array([   370,     74, 497622], dtype=int64))

### Ciro - Arvore de Decisao - tfidf

In [116]:
X_tfidf_ciro_train2 = X_tfidf_ciro_train

tree_ciro_tfidf_test = tree.DecisionTreeClassifier()
tree_X_tfidf_ciro_train, tree_X_tfidf_ciro_test, tree_Y_tfidf_ciro_train, tree_Y_tfidf_ciro_test = train_test_split(X_tfidf_ciro_train2, Y_ciro_train, test_size=0.2)
tree_ciro_tfidf_test = tree_ciro_tfidf_test.fit(tree_X_tfidf_ciro_train, tree_Y_tfidf_ciro_train)

score_tree_tfidf = tree_ciro_tfidf_test.score(tree_X_tfidf_ciro_test, tree_Y_tfidf_ciro_test)
tree_ciro_tfidf = tree_ciro_tfidf_test
for i in range(0,100):
    tree_ciro_tfidf_test = tree.DecisionTreeClassifier()
    tree_X_tfidf_ciro_train, tree_X_tfidf_ciro_test, tree_Y_tfidf_ciro_train, tree_Y_tfidf_ciro_test = train_test_split(X_tfidf_ciro_train2, Y_ciro_train, test_size=0.2)
    tree_ciro_tfidf_test = tree_ciro_tfidf_test.fit(tree_X_tfidf_ciro_train, tree_Y_tfidf_ciro_train)
    score_tree_tfidf_test = tree_ciro_tfidf_test.score(tree_X_tfidf_ciro_test, tree_Y_tfidf_ciro_test)
    if(score_tree_tfidf_test>score_tree_tfidf):
        score_tree_tfidf = score_tree_tfidf_test
        tree_ciro_tfidf = tree_ciro_tfidf_test
score_tree_tfidf

0.9985272459499264

In [117]:
i1=0
for i in range(5000,len(df_ciro),5000):
    if (i==5000):
        predict_ciro_tfidf_tree = tree_ciro_tfidf.predict(tfidf_teste(df_ciro[i1:i], tfidf_ciro).toarray())
    else:
        predict_ciro_tfidf_tree = np.append(predict_ciro_tfidf_tree, tree_ciro_tfidf.predict(tfidf_teste(df_ciro[i1:i], tfidf_ciro).toarray()))
    i1=i

predict_ciro_tfidf_tree = np.append(predict_ciro_tfidf_tree,tree_ciro_tfidf.predict(tfidf_teste(df_ciro[i1:], tfidf_ciro).toarray()))


df_ciro['predict_tfidf_ArvoreDeDecisao'] = predict_ciro_tfidf_tree

np.unique(predict_ciro_tfidf_tree, return_counts=True)

(array([-1,  0,  1]), array([   667,   1109, 496290], dtype=int64))

### Ciro - Naive Bayes - bw

In [118]:
GaussianNB_ciro_bw_test = GaussianNB()
GaussianNB_X_bw_ciro_train, GaussianNB_X_bw_ciro_test, GaussianNB_Y_bw_ciro_train, GaussianNB_Y_bw_ciro_test = train_test_split(X_bw_ciro_train2, Y_ciro_train, test_size=0.2)
GaussianNB_ciro_bw_test = GaussianNB_ciro_bw_test.fit(GaussianNB_X_bw_ciro_train.toarray(), GaussianNB_Y_bw_ciro_train)

score_GaussianNB_bw = GaussianNB_ciro_bw_test.score(GaussianNB_X_bw_ciro_test.toarray(), GaussianNB_Y_bw_ciro_test)
GaussianNB_ciro_bw = GaussianNB_ciro_bw_test
for i in range(0,100):
    GaussianNB_ciro_bw_test = GaussianNB()
    GaussianNB_X_bw_ciro_train, GaussianNB_X_bw_ciro_test, GaussianNB_Y_bw_ciro_train, GaussianNB_Y_bw_ciro_test = train_test_split(X_bw_ciro_train2, Y_ciro_train, test_size=0.2)
    GaussianNB_ciro_bw_test = GaussianNB_ciro_bw_test.fit(GaussianNB_X_bw_ciro_train.toarray(), GaussianNB_Y_bw_ciro_train)
    score_GaussianNB_bw_test = GaussianNB_ciro_bw_test.score(GaussianNB_X_bw_ciro_test.toarray(), GaussianNB_Y_bw_ciro_test)
    if(score_GaussianNB_bw_test>score_GaussianNB_bw):
        score_GaussianNB_bw = score_GaussianNB_bw_test
        GaussianNB_ciro_bw = GaussianNB_ciro_bw_test
score_GaussianNB_bw

0.9852724594992637

In [119]:
i1=0
for i in range(5000,len(df_ciro),5000):
    if (i==5000):
        predict_ciro_bw_nb = GaussianNB_ciro_bw.predict(Bag_of_words_teste(df_ciro[i1:i], bw_ciro).toarray())
    else:
        predict_ciro_bw_nb = np.append(predict_ciro_bw_nb, GaussianNB_ciro_bw.predict(Bag_of_words_teste(df_ciro[i1:i], bw_ciro).toarray()))
    i1=i

predict_ciro_bw_nb = np.append(predict_ciro_bw_nb,GaussianNB_ciro_bw.predict(Bag_of_words_teste(df_ciro[i1:], bw_ciro).toarray()))

df_ciro['predict_bw_NaiveBayes'] = predict_ciro_bw_nb

np.unique(predict_ciro_bw_nb, return_counts=True)

(array([-1,  0,  1]), array([  4282,   3316, 490468], dtype=int64))

### Ciro - Naive Bayes - tfidf

In [120]:
GaussianNB_ciro_tfidf_test = GaussianNB()
GaussianNB_X_tfidf_ciro_train, GaussianNB_X_tfidf_ciro_test, GaussianNB_Y_tfidf_ciro_train, GaussianNB_Y_tfidf_ciro_test = train_test_split(X_tfidf_ciro_train2, Y_ciro_train, test_size=0.2)
GaussianNB_ciro_tfidf_test = GaussianNB_ciro_tfidf_test.fit(GaussianNB_X_tfidf_ciro_train.toarray(), GaussianNB_Y_tfidf_ciro_train)

score_GaussianNB_tfidf = GaussianNB_ciro_tfidf_test.score(GaussianNB_X_tfidf_ciro_test.toarray(), GaussianNB_Y_tfidf_ciro_test)
GaussianNB_ciro_tfidf = GaussianNB_ciro_tfidf_test
for i in range(0,100):
    GaussianNB_ciro_tfidf_test = GaussianNB()
    GaussianNB_X_tfidf_ciro_train, GaussianNB_X_tfidf_ciro_test, GaussianNB_Y_tfidf_ciro_train, GaussianNB_Y_tfidf_ciro_test = train_test_split(X_tfidf_ciro_train2, Y_ciro_train, test_size=0.2)
    GaussianNB_ciro_tfidf_test = GaussianNB_ciro_tfidf_test.fit(GaussianNB_X_tfidf_ciro_train.toarray(), GaussianNB_Y_tfidf_ciro_train)
    score_GaussianNB_tfidf_test = GaussianNB_ciro_tfidf_test.score(GaussianNB_X_tfidf_ciro_test.toarray(), GaussianNB_Y_tfidf_ciro_test)
    if(score_GaussianNB_tfidf_test>score_GaussianNB_tfidf):
        score_GaussianNB_tfidf = score_GaussianNB_tfidf_test
        GaussianNB_ciro_tfidf = GaussianNB_ciro_tfidf_test
score_GaussianNB_tfidf

0.9845360824742269

In [121]:
i1=0
for i in range(5000,len(df_ciro),5000):
    if (i==5000):
        predict_ciro_tfidf_nb = GaussianNB_ciro_tfidf.predict(tfidf_teste(df_ciro[i1:i], tfidf_ciro).toarray())
    else:
        predict_ciro_tfidf_nb = np.append(predict_ciro_tfidf_nb, GaussianNB_ciro_tfidf.predict(tfidf_teste(df_ciro[i1:i], tfidf_ciro).toarray()))
    i1=i

predict_ciro_tfidf_nb = np.append(predict_ciro_tfidf_nb,GaussianNB_ciro_tfidf.predict(tfidf_teste(df_ciro[i1:], tfidf_ciro).toarray()))

df_ciro['predict_tfidf_NaiveBayes'] = predict_ciro_tfidf_nb

np.unique(predict_ciro_tfidf_nb, return_counts=True)

(array([-1,  0,  1]), array([  4672,   1782, 491612], dtype=int64))

### Ciro - Random Forest - bw

In [122]:
RandomForest_ciro_bw_test = RandomForestClassifier()
RandomForest_X_bw_ciro_train, RandomForest_X_bw_ciro_test, RandomForest_Y_bw_ciro_train, RandomForest_Y_bw_ciro_test = train_test_split(X_bw_ciro_train2, Y_ciro_train, test_size=0.2)
RandomForest_ciro_bw_test = RandomForest_ciro_bw_test.fit(RandomForest_X_bw_ciro_train.toarray(), RandomForest_Y_bw_ciro_train)

score_RandomForest_bw = RandomForest_ciro_bw_test.score(RandomForest_X_bw_ciro_test.toarray(), RandomForest_Y_bw_ciro_test)
RandomForest_ciro_bw = RandomForest_ciro_bw_test
for i in range(0,100):
    RandomForest_ciro_bw_test = RandomForestClassifier()
    RandomForest_X_bw_ciro_train, RandomForest_X_bw_ciro_test, RandomForest_Y_bw_ciro_train, RandomForest_Y_bw_ciro_test = train_test_split(X_bw_ciro_train2, Y_ciro_train, test_size=0.2)
    RandomForest_ciro_bw_test = RandomForest_ciro_bw_test.fit(RandomForest_X_bw_ciro_train.toarray(), RandomForest_Y_bw_ciro_train)
    score_RandomForest_bw_test = RandomForest_ciro_bw_test.score(RandomForest_X_bw_ciro_test.toarray(), RandomForest_Y_bw_ciro_test)
    if(score_RandomForest_bw_test>score_RandomForest_bw):
        score_RandomForest_bw = score_RandomForest_bw_test
        RandomForest_ciro_bw = RandomForest_ciro_bw_test
score_RandomForest_bw

0.9992636229749632

In [123]:
i1=0
for i in range(5000,len(df_ciro),5000):
    if (i==5000):
        predict_ciro_bw_rf = RandomForest_ciro_bw.predict(Bag_of_words_teste(df_ciro[i1:i], bw_ciro).toarray())
    else:
        predict_ciro_bw_rf = np.append(predict_ciro_bw_rf, RandomForest_ciro_bw.predict(Bag_of_words_teste(df_ciro[i1:i], bw_ciro).toarray()))
    i1=i

predict_ciro_bw_rf = np.append(predict_ciro_bw_rf,RandomForest_ciro_bw.predict(Bag_of_words_teste(df_ciro[i1:], bw_ciro).toarray()))

df_ciro['predict_bw_RandomForest'] = predict_ciro_bw_rf

np.unique(predict_ciro_bw_rf, return_counts=True)

(array([-1,  0,  1]), array([    12,     11, 498043], dtype=int64))

### Ciro - Random Forest - tfidf

In [124]:
RandomForest_ciro_tfidf_test = RandomForestClassifier()
RandomForest_X_tfidf_ciro_train, RandomForest_X_tfidf_ciro_test, RandomForest_Y_tfidf_ciro_train, RandomForest_Y_tfidf_ciro_test = train_test_split(X_tfidf_ciro_train2, Y_ciro_train, test_size=0.2)
RandomForest_ciro_tfidf_test = RandomForest_ciro_tfidf_test.fit(RandomForest_X_tfidf_ciro_train.toarray(), RandomForest_Y_tfidf_ciro_train)

score_RandomForest_tfidf = RandomForest_ciro_tfidf_test.score(RandomForest_X_tfidf_ciro_test.toarray(), RandomForest_Y_tfidf_ciro_test)
RandomForest_ciro_tfidf = RandomForest_ciro_tfidf_test
for i in range(0,100):
    RandomForest_ciro_tfidf_test = RandomForestClassifier()
    RandomForest_X_tfidf_ciro_train, RandomForest_X_tfidf_ciro_test, RandomForest_Y_tfidf_ciro_train, RandomForest_Y_tfidf_ciro_test = train_test_split(X_tfidf_ciro_train2, Y_ciro_train, test_size=0.2)
    RandomForest_ciro_tfidf_test = RandomForest_ciro_tfidf_test.fit(RandomForest_X_tfidf_ciro_train.toarray(), RandomForest_Y_tfidf_ciro_train)
    score_RandomForest_tfidf_test = RandomForest_ciro_tfidf_test.score(RandomForest_X_tfidf_ciro_test.toarray(), RandomForest_Y_tfidf_ciro_test)
    if(score_RandomForest_tfidf_test>score_RandomForest_tfidf):
        score_RandomForest_tfidf = score_RandomForest_tfidf_test
        RandomForest_ciro_tfidf = RandomForest_ciro_tfidf_test
score_RandomForest_tfidf

0.9992636229749632

In [125]:
i1=0
for i in range(5000,len(df_ciro),5000):
    if (i==5000):
        predict_ciro_tfidf_rf = RandomForest_ciro_tfidf.predict(tfidf_teste(df_ciro[i1:i], tfidf_ciro).toarray())
    else:
        predict_ciro_tfidf_rf = np.append(predict_ciro_tfidf_rf, RandomForest_ciro_tfidf.predict(tfidf_teste(df_ciro[i1:i], tfidf_ciro).toarray()))
    i1=i

predict_ciro_tfidf_rf = np.append(predict_ciro_tfidf_rf,RandomForest_ciro_tfidf.predict(tfidf_teste(df_ciro[i1:], tfidf_ciro).toarray()))

df_ciro['predict_tfidf_RandomForest'] = predict_ciro_tfidf_rf

np.unique(predict_ciro_tfidf_rf, return_counts=True)

(array([-1,  0,  1]), array([    12,     14, 498040], dtype=int64))

### Ciro - SVM - bw

In [126]:
svc_ciro_bw_test = LinearSVC()
svc_X_bw_ciro_train, svc_X_bw_ciro_test, svc_Y_bw_ciro_train, svc_Y_bw_ciro_test = train_test_split(X_bw_ciro_train2, Y_ciro_train, test_size=0.2)
svc_ciro_bw_test = svc_ciro_bw_test.fit(svc_X_bw_ciro_train.toarray(), svc_Y_bw_ciro_train)

score_svc_bw = svc_ciro_bw_test.score(svc_X_bw_ciro_test.toarray(), svc_Y_bw_ciro_test)
svc_ciro_bw = svc_ciro_bw_test
for i in range(0,100):
    svc_ciro_bw_test = LinearSVC()
    svc_X_bw_ciro_train, svc_X_bw_ciro_test, svc_Y_bw_ciro_train, svc_Y_bw_ciro_test = train_test_split(X_bw_ciro_train2, Y_ciro_train, test_size=0.2)
    svc_ciro_bw_test = svc_ciro_bw_test.fit(svc_X_bw_ciro_train.toarray(), svc_Y_bw_ciro_train)
    score_svc_bw_test = svc_ciro_bw_test.score(svc_X_bw_ciro_test.toarray(), svc_Y_bw_ciro_test)
    if(score_svc_bw_test>score_svc_bw):
        score_svc_bw = score_svc_bw_test
        svc_ciro_bw = svc_ciro_bw_test
score_svc_bw

C:\Users\daian\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\daian\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\daian\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


0.9985272459499264

In [127]:
i1=0
for i in range(5000,len(df_ciro),5000):
    if (i==5000):
        predict_ciro_bw_svc = svc_ciro_bw.predict(Bag_of_words_teste(df_ciro[i1:i], bw_ciro).toarray())
    else:
        predict_ciro_bw_svc = np.append(predict_ciro_bw_svc, svc_ciro_bw.predict(Bag_of_words_teste(df_ciro[i1:i], bw_ciro).toarray()))
    i1=i

predict_ciro_bw_svc = np.append(predict_ciro_bw_svc,svc_ciro_bw.predict(Bag_of_words_teste(df_ciro[i1:], bw_ciro).toarray()))

df_ciro['predict_bw_svc'] = predict_ciro_bw_svc

np.unique(predict_ciro_bw_svc, return_counts=True)

(array([-1,  0,  1]), array([    75,     21, 497970], dtype=int64))

### Ciro - SVM - tfidf

In [128]:
svc_ciro_tfidf_test = LinearSVC()
svc_X_tfidf_ciro_train, svc_X_tfidf_ciro_test, svc_Y_tfidf_ciro_train, svc_Y_tfidf_ciro_test = train_test_split(X_tfidf_ciro_train2, Y_ciro_train, test_size=0.2)
svc_ciro_tfidf_test = svc_ciro_tfidf_test.fit(svc_X_tfidf_ciro_train.toarray(), svc_Y_tfidf_ciro_train)

score_svc_tfidf = svc_ciro_tfidf_test.score(svc_X_tfidf_ciro_test.toarray(), svc_Y_tfidf_ciro_test)
svc_ciro_tfidf = svc_ciro_tfidf_test
for i in range(0,100):
    svc_ciro_tfidf_test = LinearSVC()
    svc_X_tfidf_ciro_train, svc_X_tfidf_ciro_test, svc_Y_tfidf_ciro_train, svc_Y_tfidf_ciro_test = train_test_split(X_tfidf_ciro_train2, Y_ciro_train, test_size=0.2)
    svc_ciro_tfidf_test = svc_ciro_tfidf_test.fit(svc_X_tfidf_ciro_train.toarray(), svc_Y_tfidf_ciro_train)
    score_svc_tfidf_test = svc_ciro_tfidf_test.score(svc_X_tfidf_ciro_test.toarray(), svc_Y_tfidf_ciro_test)
    if(score_svc_tfidf_test>score_svc_tfidf):
        score_svc_tfidf = score_svc_tfidf_test
        svc_ciro_tfidf = svc_ciro_tfidf_test
score_svc_tfidf

0.9985272459499264

In [129]:
i1=0
for i in range(5000,len(df_ciro),5000):
    if (i==5000):
        predict_ciro_tfidf_svc = svc_ciro_tfidf.predict(tfidf_teste(df_ciro[i1:i], tfidf_ciro).toarray())
    else:
        predict_ciro_tfidf_svc = np.append(predict_ciro_tfidf_svc, svc_ciro_tfidf.predict(tfidf_teste(df_ciro[i1:i], tfidf_ciro).toarray()))
    i1=i

predict_ciro_tfidf_svc = np.append(predict_ciro_tfidf_svc,svc_ciro_tfidf.predict(tfidf_teste(df_ciro[i1:], tfidf_ciro).toarray()))

df_ciro['predict_tfidf_svc'] = predict_ciro_tfidf_svc

np.unique(predict_ciro_tfidf_svc, return_counts=True)

(array([-1,  0,  1]), array([    17,      8, 498041], dtype=int64))

### Ciro - MLP - bw

In [130]:
mlp_ciro_bw_test = MLPClassifier()
mlp_X_bw_ciro_train, mlp_X_bw_ciro_test, mlp_Y_bw_ciro_train, mlp_Y_bw_ciro_test = train_test_split(X_bw_ciro_train2, Y_ciro_train, test_size=0.2)
mlp_ciro_bw_test = mlp_ciro_bw_test.fit(mlp_X_bw_ciro_train.toarray(), mlp_Y_bw_ciro_train)

score_mlp_bw = mlp_ciro_bw_test.score(mlp_X_bw_ciro_test.toarray(), mlp_Y_bw_ciro_test)
mlp_ciro_bw = mlp_ciro_bw_test
for i in range(0,10):
    mlp_ciro_bw_test = MLPClassifier()
    mlp_X_bw_ciro_train, mlp_X_bw_ciro_test, mlp_Y_bw_ciro_train, mlp_Y_bw_ciro_test = train_test_split(X_bw_ciro_train2, Y_ciro_train, test_size=0.2)
    mlp_ciro_bw_test = mlp_ciro_bw_test.fit(mlp_X_bw_ciro_train.toarray(), mlp_Y_bw_ciro_train)
    score_mlp_bw_test = mlp_ciro_bw_test.score(mlp_X_bw_ciro_test.toarray(), mlp_Y_bw_ciro_test)
    if(score_mlp_bw_test>score_mlp_bw):
        score_mlp_bw = score_mlp_bw_test
        mlp_ciro_bw = mlp_ciro_bw_test
score_mlp_bw

0.9985272459499264

In [131]:
i1=0
for i in range(5000,len(df_ciro),5000):
    if (i==5000):
        predict_ciro_bw_mlp = mlp_ciro_bw.predict(Bag_of_words_teste(df_ciro[i1:i], bw_ciro).toarray())
    else:
        predict_ciro_bw_mlp = np.append(predict_ciro_bw_mlp, mlp_ciro_bw.predict(Bag_of_words_teste(df_ciro[i1:i], bw_ciro).toarray()))
    i1=i

predict_ciro_bw_mlp = np.append(predict_ciro_bw_mlp,mlp_ciro_bw.predict(Bag_of_words_teste(df_ciro[i1:], bw_ciro).toarray()))

df_ciro['predict_bw_mlp'] = predict_ciro_bw_mlp
df_ciro['predict_bw_mlp2'] = predict_ciro_bw_mlp

np.unique(predict_ciro_bw_mlp, return_counts=True)

(array([-1,  0,  1]), array([    17,     14, 498035], dtype=int64))

### Ciro - MLP - tfidf

In [132]:
mlp_ciro_tfidf_test = MLPClassifier()
mlp_X_tfidf_ciro_train, mlp_X_tfidf_ciro_test, mlp_Y_tfidf_ciro_train, mlp_Y_tfidf_ciro_test = train_test_split(X_tfidf_ciro_train2, Y_ciro_train, test_size=0.2)
mlp_ciro_tfidf_test = mlp_ciro_tfidf_test.fit(mlp_X_tfidf_ciro_train.toarray(), mlp_Y_tfidf_ciro_train)

score_mlp_tfidf = mlp_ciro_tfidf_test.score(mlp_X_tfidf_ciro_test.toarray(), mlp_Y_tfidf_ciro_test)
mlp_ciro_tfidf = mlp_ciro_tfidf_test
for i in range(0,10):
    mlp_ciro_tfidf_test = MLPClassifier()
    mlp_X_tfidf_ciro_train, mlp_X_tfidf_ciro_test, mlp_Y_tfidf_ciro_train, mlp_Y_tfidf_ciro_test = train_test_split(X_tfidf_ciro_train2, Y_ciro_train, test_size=0.2)
    mlp_ciro_tfidf_test = mlp_ciro_tfidf_test.fit(mlp_X_tfidf_ciro_train.toarray(), mlp_Y_tfidf_ciro_train)
    score_mlp_tfidf_test = mlp_ciro_tfidf_test.score(mlp_X_tfidf_ciro_test.toarray(), mlp_Y_tfidf_ciro_test)
    if(score_mlp_tfidf_test>score_mlp_tfidf):
        score_mlp_tfidf = score_mlp_tfidf_test
        mlp_ciro_tfidf = mlp_ciro_tfidf_test
score_mlp_tfidf

0.9992636229749632

In [133]:
i1=0
for i in range(5000,len(df_ciro),5000):
    if (i==5000):
        predict_ciro_tfidf_mlp = mlp_ciro_tfidf.predict(tfidf_teste(df_ciro[i1:i], tfidf_ciro).toarray())
    else:
        predict_ciro_tfidf_mlp = np.append(predict_ciro_tfidf_mlp, mlp_ciro_tfidf.predict(tfidf_teste(df_ciro[i1:i], tfidf_ciro).toarray()))
    i1=i

predict_ciro_tfidf_mlp = np.append(predict_ciro_tfidf_mlp,mlp_ciro_tfidf.predict(tfidf_teste(df_ciro[i1:], tfidf_ciro).toarray()))

df_ciro['predict_tfidf_mlp'] = predict_ciro_tfidf_mlp
df_ciro['predict_tfidf_mlp2'] = predict_ciro_tfidf_mlp

np.unique(predict_ciro_tfidf_mlp, return_counts=True)

(array([-1,  0,  1]), array([    22,     14, 498030], dtype=int64))

In [134]:
df_bolsonaro.to_excel('bolsonaro_predict2.xlsx')
df_lula.to_excel('lula_predict2.xlsx')
df_simone.to_excel('simone_predict2.xlsx')
df_ciro.to_excel('ciro_predict2.xlsx')

In [135]:
def VotacaoPredict(df):
    for i in df.index:
        pos=0
        neu=0
        neg=0
        for j in ['predict_tfidf_mlp','predict_bw_mlp',
                  'predict_tfidf_svc','predict_bw_svc',
                 'predict_tfidf_RandomForest','predict_bw_RandomForest',
                 'predict_tfidf_NaiveBayes','predict_bw_NaiveBayes',
                 'predict_tfidf_ArvoreDeDecisao','predict_bw_ArvoreDeDecisao']:
            if (df.loc[i,j]==1):
                pos+=1
            elif (df.loc[i,j]==0):
                neu+=1    
            else:
                neg+=1
            
        
        if((neu>=neg)& (neu>=pos)):
            df.loc[i,'VotacaoPredict_norm'] = 0
        elif((neg>=pos)& (neg>=neu)):
            df.loc[i,'VotacaoPredict_norm'] = -1
        else:
            df.loc[i,'VotacaoPredict_norm'] = 1     
    print(df['VotacaoPredict_norm'].value_counts())        
    return df

In [136]:
df_bolsonaro_predict2 = VotacaoPredict(df_bolsonaro)
df_lula_predict2 = VotacaoPredict(df_lula)
df_simone_predict2 = VotacaoPredict(df_simone)
df_ciro_predict2 = VotacaoPredict(df_ciro)

 1.0    732672
-1.0    209248
 0.0      4518
Name: VotacaoPredict_ajust, dtype: int64
 1.0    877772
-1.0     65899
 0.0      2767
Name: VotacaoPredict_norm, dtype: int64
 1.0    741160
-1.0    190450
 0.0       120
Name: VotacaoPredict_ajust, dtype: int64
 1.0    874608
-1.0     57094
 0.0        28
Name: VotacaoPredict_norm, dtype: int64
 0.0    84932
 1.0    39112
-1.0     7118
Name: VotacaoPredict_ajust, dtype: int64
 1.0    65021
 0.0    63718
-1.0     2423
Name: VotacaoPredict_norm, dtype: int64
 1.0    498011
-1.0        36
 0.0        19
Name: VotacaoPredict_ajust, dtype: int64
 1.0    498033
-1.0        20
 0.0        13
Name: VotacaoPredict_norm, dtype: int64


In [86]:
print(df_bolsonaro_predict['VotacaoPredict_norm'].value_counts())
print(df_lula_predict['VotacaoPredict_norm'].value_counts())
print(df_simone_predict['VotacaoPredict_norm'].value_counts())
print(df_ciro_predict['VotacaoPredict_norm'].value_counts())

 1.0    826633
-1.0    116816
 0.0      2989
Name: VotacaoPredict_norm, dtype: int64
 1.0    842343
-1.0     89340
 0.0        47
Name: VotacaoPredict_norm, dtype: int64
 0.0    73303
 1.0    52873
-1.0     4986
Name: VotacaoPredict_norm, dtype: int64
 1.0    498033
-1.0        20
 0.0        13
Name: VotacaoPredict_norm, dtype: int64


In [137]:
df_bolsonaro_predict2.to_excel('bolsonaro_resultado_treinoTeste.xlsx')
df_lula_predict2.to_excel('lula_resultado_treinoTeste.xlsx')
df_simone_predict2.to_excel('simone_resultado_treinoTeste.xlsx')
df_ciro_predict2.to_excel('ciro_resultado_treinoTeste.xlsx')